# 🔍 AML Investigation Agent (Qwen3-30B-A3B-Instruct + LangGraph)

An autonomous AI Agent for investigating financial transaction graphs to identify money laundering patterns using **Qwen3-30B-A3B-Instruct-2507** (Mixture of Experts) and **LangGraph** stateful orchestration.

## Architecture Overview

| Component | Technology | Description |
|-----------|------------|-------------|
| **Base Model** | Qwen3-30B-A3B-Instruct-2507 | MOE architecture with native tool calling (faster than Thinking) |
| **Fine-Tuning** | Unsloth FastModel + LoRA | 4-bit quantization, 2x faster training (MOE optimized) |
| **Data Processing** | Polars | High-performance dataframes |
| **Graph Analysis** | NetworkX | Transaction network traversal |
| **Agent Framework** | LangGraph + MemorySaver | Stateful graph with conditional routing |
| **RL Training** | TRL GRPOTrainer | Group Relative Policy Optimization |
| **Observability** | MLflow Tracing | Automatic LangGraph trace logging |
| **Evaluation** | Gemini LLM-as-Judge | Strategy quality scoring |

## LangGraph Agent Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                        MemorySaver                              │
│              (Persistent State Across Loop)                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│    ┌──────────────┐      ┌──────────────────┐                   │
│    │              │      │   Conditional    │                   │
│    │  Agent Node  │─────▶│     Edge         │                   │
│    │  (Brain)     │      │   (Router)       │                   │
│    │              │      │                  │                   │
│    └──────────────┘      └────────┬─────────┘                   │
│          ▲                        │                             │
│          │                        ▼                             │
│          │         ┌──────────────────────────┐                 │
│          │         │  continue?  │  report?   │                 │
│          │         └──────┬──────┴─────┬──────┘                 │
│          │                │            │                        │
│          │                ▼            ▼                        │
│    ┌─────┴──────┐   ┌──────────┐  ┌─────────┐                   │
│    │            │   │  Tool    │  │  END    │                   │
│    │  (loop)    │◀──│  Node    │  │  Node   │                   │
│    │            │   │          │  │         │                   │
│    └────────────┘   └──────────┘  └─────────┘                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## Investigation Tools

| Tool | Description |
|------|-------------|
| `get_account_summary` | Get account metadata and risk assessment |
| `get_recent_transactions` | Get top-5 recent transaction flows |
| `check_sanctions_list` | Verify against OFAC watchlist |
| `submit_sar` | **Terminal action** - Submit Suspicious Activity Report |

## Win Condition
`submit_sar` on an entity that is **both sanctioned AND reachable via a laundering path** from the seed account.


## 1. Setup & Configuration


In [ ]:
# ============================================================================
# AML INVESTIGATION AGENT (LangGraph) - Setup & Dependencies
# ============================================================================

# CRITICAL: Import Unsloth FIRST before any other ML libraries
# This ensures all optimizations are applied correctly
# NOTE: For MOE models (like Qwen3-30B-A3B), use FastModel instead of FastLanguageModel
#       See: https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune/qwen3-2507
from unsloth import FastModel

import os
import sys
import json
import random
import time
import re
import uuid
from datetime import datetime
from dataclasses import dataclass, field
from typing import Dict, List, Any, Tuple, Optional, Annotated, Literal, TypedDict, Sequence
import operator
from pathlib import Path

# Numerical & Data Processing
import numpy as np
import polars as pl
import pandas as pd
import networkx as nx

# ML & Deep Learning (imported AFTER unsloth)
import torch

# Environment
from dotenv import load_dotenv
load_dotenv()

# ============================================================================
# CONFIGURATION
# ============================================================================

# Model Configuration
# Available Qwen3-2507 models (MOE = Mixture of Experts):
#   - "unsloth/Qwen3-4B-Thinking-2507"                    # ~3GB VRAM - Dense, small
#   - "unsloth/Qwen3-30B-A3B-Thinking-2507"               # ~17GB VRAM - MOE (Thinking mode)
#   - "unsloth/Qwen3-30B-A3B-Instruct-2507"               # ~17GB VRAM - MOE (Instruct mode)
#   - "unsloth/Qwen3-235B-A22B-Thinking-2507"             # Multi-GPU - MOE (Thinking mode)
#   - "unsloth/Qwen3-235B-A22B-Instruct-2507"             # Multi-GPU - MOE (Instruct mode)
#
# IMPORTANT: MOE models require FastModel (not FastLanguageModel)
# Thinking models: use temperature=0.6, top_p=0.95
# Instruct models: use temperature=0.7, top_p=0.8
# IMPORTANT: Use Instruct model for faster inference with tool calling
# Thinking models are VERY slow because they generate long reasoning traces
# Instruct models are designed for fast, direct responses with tool calling support
MODEL_NAME = "unsloth/Qwen3-30B-A3B-Instruct-2507"
IS_THINKING_MODEL = "Thinking" in MODEL_NAME  # Auto-detect thinking vs instruct
GEMINI_MODEL = "gemini-2.0-flash"    # LLM-as-Judge

# Qwen3-2507 Recommended Generation Settings (from Unsloth docs)
# https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune/qwen3-2507
if IS_THINKING_MODEL:
    # Thinking model settings
    GENERATION_TEMPERATURE = 0.6
    GENERATION_TOP_P = 0.95
    GENERATION_TOP_K = 20
    GENERATION_MIN_P = 0.0
else:
    # Instruct model settings
    GENERATION_TEMPERATURE = 0.7
    GENERATION_TOP_P = 0.8
    GENERATION_TOP_K = 20
    GENERATION_MIN_P = 0.0

# Agent Configuration
MAX_STEPS = 50                        # Max steps per investigation
MAX_HISTORY_TURNS = 6                 # Conversation history limit

# Training Configuration
SFT_EPOCHS = 3
SFT_LEARNING_RATE = 2e-4
GRPO_EPOCHS = 1
GRPO_LEARNING_RATE = 5e-6

# LoRA Configuration
LORA_R = 32                           # Higher rank for complex reasoning
LORA_ALPHA = 64
LORA_TARGET_MODULES = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]

# Evaluation Configuration
EVAL_EPISODES = 10                    # Episodes per evaluation stage

# Random Seed
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Paths (relative to notebook location)
NOTEBOOK_DIR = Path(".").resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
DATA_DIR = PROJECT_ROOT / "data" / "raw"
MODELS_DIR = PROJECT_ROOT / "models"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

# Ensure directories exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Dataset Selection
DATASET_SIZE = "Small"   # Options: "Small", "Medium", "Large"
DATASET_PREFIX = "LI"    # Options: "LI" (Low Illicit), "HI" (High Illicit)

# ============================================================================
# RESULTS STORAGE - For comparison across training stages
# ============================================================================

evaluation_results = {
    "baseline": None,
    "post_sft": None,
    "post_grpo": None,
}

print("=" * 60)
print("🔍 AML INVESTIGATION AGENT (LangGraph) - Configuration")
print("=" * 60)
print(f"  Model:           {MODEL_NAME}")
print(f"  Judge:           {GEMINI_MODEL}")
print(f"  Dataset:         {DATASET_PREFIX}-{DATASET_SIZE}")
print(f"  Max Steps:       {MAX_STEPS}")
print(f"  Eval Episodes:   {EVAL_EPISODES}")
print(f"  LoRA Rank:       {LORA_R}")
print(f"  LoRA Alpha:      {LORA_ALPHA}")
print(f"  Random Seed:     {RANDOM_SEED}")
print(f"  Project Root:    {PROJECT_ROOT}")
print(f"  Data Dir:        {DATA_DIR}")
print(f"  Models Dir:      {MODELS_DIR}")
print(f"  GPU Available:   {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU Device:      {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory:      {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"  Thinking Model:  {IS_THINKING_MODEL}")
print(f"  Temperature:     {GENERATION_TEMPERATURE}")
print(f"  Top-P:           {GENERATION_TOP_P}")
print(f"  Top-K:           {GENERATION_TOP_K}")
print("=" * 60)


## 2. Data Loading with Polars

Download the IBM AML dataset from Kaggle (if not already present) and load using Polars for high-performance data manipulation.

**Dataset**: [IBM Transactions for Anti Money Laundering (AML)](https://www.kaggle.com/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml/data)


In [ ]:
# ============================================================================
# DOWNLOAD DATASET FROM KAGGLE - IBM AML Transactions Dataset
# ============================================================================

import zipfile
import shutil

# Kaggle dataset identifier
KAGGLE_DATASET = "ealtman2019/ibm-transactions-for-anti-money-laundering-aml"

# Check if required files exist
trans_file = DATA_DIR / f"{DATASET_PREFIX}-{DATASET_SIZE}_Trans.csv"
accounts_file = DATA_DIR / f"{DATASET_PREFIX}-{DATASET_SIZE}_accounts.csv"
patterns_file = DATA_DIR / f"{DATASET_PREFIX}-{DATASET_SIZE}_Patterns.txt"

files_exist = trans_file.exists() and accounts_file.exists() and patterns_file.exists()

if files_exist:
    print(f"✓ Dataset already exists at {DATA_DIR}")
    print(f"  - Transactions: {trans_file.name}")
    print(f"  - Accounts: {accounts_file.name}")
    print(f"  - Patterns: {patterns_file.name}")
else:
    print(f"📥 Dataset not found. Downloading from Kaggle...")
    print(f"   Dataset: {KAGGLE_DATASET}")
    
    # Ensure data directory exists
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    
    try:
        # Import and authenticate Kaggle API
        from kaggle.api.kaggle_api_extended import KaggleApi
        
        api = KaggleApi()
        api.authenticate()
        
        print(f"   ✓ Kaggle API authenticated")
        
        # Download dataset
        print(f"   Downloading dataset to {DATA_DIR}...")
        api.dataset_download_files(
            dataset=KAGGLE_DATASET,
            path=str(DATA_DIR),
            unzip=True,
            quiet=False
        )
        
        print(f"   ✓ Download complete!")
        
        # List downloaded files
        print(f"\n   Downloaded files:")
        for f in sorted(DATA_DIR.glob("*")):
            size_mb = f.stat().st_size / (1024 * 1024)
            print(f"     - {f.name} ({size_mb:.1f} MB)")
        
    except ImportError:
        print("   ❌ Kaggle package not installed.")
        print("   Run: pip install kaggle")
        print("   Then set up ~/.kaggle/kaggle.json with your API credentials")
        raise ImportError("Please install kaggle package: pip install kaggle")
        
    except Exception as e:
        print(f"   ❌ Error downloading dataset: {e}")
        print("\n   Manual download instructions:")
        print(f"   1. Visit: https://www.kaggle.com/datasets/{KAGGLE_DATASET}")
        print(f"   2. Download and extract to: {DATA_DIR}")
        raise

# Verify files exist after download
assert trans_file.exists(), f"Transaction file not found: {trans_file}"
assert accounts_file.exists(), f"Accounts file not found: {accounts_file}"
assert patterns_file.exists(), f"Patterns file not found: {patterns_file}"

print(f"\n{'=' * 60}")
print(f"✓ DATASET READY: {DATASET_PREFIX}-{DATASET_SIZE}")
print(f"{'=' * 60}")


In [ ]:
# ============================================================================
# DATA LOADING - IBM AML Dataset with Polars
# ============================================================================
# FIX: The transactions file has LEADING ZEROS on bank IDs (e.g., "001120")
# but the accounts file stores them without (e.g., "1120"). We normalize by
# casting bank IDs to Int64 first, which strips leading zeros.

print("📊 Loading IBM AML Dataset with Polars...")

# Load transactions
raw_trans_pl = pl.read_csv(
    trans_file,
    new_columns=[
        'timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account',
        'amount_received', 'receiving_currency', 'amount_paid',
        'payment_currency', 'payment_format', 'is_laundering'
    ],
    skip_rows=1
)
print(f"✓ Loaded {len(raw_trans_pl):,} transactions")

# Load accounts
raw_accounts_pl = pl.read_csv(accounts_file)
print(f"✓ Loaded {len(raw_accounts_pl):,} accounts")

# Process transactions - Create unique account IDs
# CRITICAL FIX: Cast bank IDs to Int64 first to remove leading zeros, then to Utf8
transactions_pl = raw_trans_pl.with_columns([
    (pl.col('from_bank').cast(pl.Int64).cast(pl.Utf8) + '-' + pl.col('from_account').cast(pl.Utf8)).alias('from_account_id'),
    (pl.col('to_bank').cast(pl.Int64).cast(pl.Utf8) + '-' + pl.col('to_account').cast(pl.Utf8)).alias('to_account_id'),
    (pl.lit('TXN-') + pl.arange(0, pl.len()).cast(pl.Utf8)).alias('transaction_id'),
    pl.col('is_laundering').cast(pl.Int32),
]).select([
    'transaction_id',
    pl.col('from_account_id').alias('from_account'),
    pl.col('to_account_id').alias('to_account'),
    pl.col('amount_received').alias('amount'),
    pl.col('receiving_currency').alias('currency'),
    'timestamp', 'is_laundering', 'payment_format',
])

# Identify laundering destinations for sanctioned marking
laundering_dests = set(
    transactions_pl.filter(pl.col('is_laundering') == 1)['to_account'].unique().to_list()
)
print(f"✓ Found {len(laundering_dests):,} laundering destination accounts")

# Process accounts - Add risk scores and sanctioned flags
# Bank ID in accounts file is already without leading zeros, so just cast to Utf8
accounts_pl = raw_accounts_pl.rename({
    'Bank Name': 'bank_name', 'Bank ID': 'bank_id',
    'Account Number': 'account_number', 'Entity ID': 'entity_id', 'Entity Name': 'entity_name'
}).with_columns([
    (pl.col('bank_id').cast(pl.Utf8) + '-' + pl.col('account_number').cast(pl.Utf8)).alias('account_id'),
    pl.when(pl.col('entity_name').str.contains('Corporation')).then(pl.lit('Corporate'))
        .when(pl.col('entity_name').str.contains('Partnership')).then(pl.lit('Partnership'))
        .when(pl.col('entity_name').str.contains('Sole Proprietorship')).then(pl.lit('Individual'))
        .otherwise(pl.lit('Unknown')).alias('account_type'),
])

# Add sanctioned flag (30% of laundering destinations) and risk scores
account_ids = accounts_pl['account_id'].to_list()
accounts_pl = accounts_pl.with_columns([
    pl.Series('is_sanctioned', [acc in laundering_dests and random.random() < 0.3 for acc in account_ids]),
    pl.Series('risk_score', [round(random.uniform(0.1, 0.9), 2) for _ in range(len(account_ids))]),
]).select(['account_id', 'bank_id', 'bank_name', 'entity_id', 'entity_name', 'account_type', 'risk_score', 'is_sanctioned'])

# Convert to pandas for NetworkX
transactions_df = transactions_pl.to_pandas()
accounts_df = accounts_pl.to_pandas()

# Summary
n_accounts, n_transactions = len(accounts_df), len(transactions_df)
n_laundering = int(transactions_df['is_laundering'].sum())
n_sanctioned = int(accounts_df['is_sanctioned'].sum())

print(f"\n{'=' * 60}")
print(f"📊 DATASET SUMMARY: {DATASET_PREFIX}-{DATASET_SIZE}")
print(f"{'=' * 60}")
print(f"  Accounts:             {n_accounts:>12,}")
print(f"  Transactions:         {n_transactions:>12,}")
print(f"  Laundering Txns:      {n_laundering:>12,} ({n_laundering/n_transactions*100:.2f}%)")
print(f"  Sanctioned Accounts:  {n_sanctioned:>12,}")
print(f"  Laundering Dests:     {len(laundering_dests):>12,}")
print(f"{'=' * 60}")


In [ ]:
# ============================================================================
# PARSE LAUNDERING PATTERNS - Extract Pattern Seeds for Training
# ============================================================================

@dataclass
class LaunderingPattern:
    """Represents a single money laundering pattern from the dataset."""
    pattern_type: str
    pattern_info: str
    transactions: List[dict]
    accounts_involved: set
    
    @property
    def seed_account(self) -> str:
        return self.transactions[0].get('from_account', '') if self.transactions else ''
    
    @property
    def terminal_account(self) -> str:
        return self.transactions[-1].get('to_account', '') if self.transactions else ''
    
    @property
    def total_amount(self) -> float:
        return sum(t.get('amount', 0) for t in self.transactions)
    
    @property
    def hop_count(self) -> int:
        return len(self.transactions)


def parse_patterns_file(filepath: Path) -> List[LaunderingPattern]:
    """Parse patterns file to extract laundering patterns."""
    patterns = []
    current_pattern = None
    
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            
            if line.startswith('BEGIN LAUNDERING ATTEMPT'):
                match = re.match(r'BEGIN LAUNDERING ATTEMPT - (\w+(?:-\w+)?):?\s*(.*)', line)
                if match:
                    current_pattern = LaunderingPattern(
                        pattern_type=match.group(1),
                        pattern_info=match.group(2).strip() if match.group(2) else "",
                        transactions=[], accounts_involved=set()
                    )
            
            elif line.startswith('END LAUNDERING ATTEMPT'):
                if current_pattern and current_pattern.transactions:
                    patterns.append(current_pattern)
                current_pattern = None
            
            elif current_pattern and line and not line.startswith('BEGIN') and not line.startswith('END'):
                parts = line.split(',')
                if len(parts) >= 7:
                    try:
                        # CRITICAL FIX: Normalize bank IDs by converting to int first
                        # This removes leading zeros (e.g., "001120" -> "1120")
                        from_bank = str(int(parts[1].strip()))
                        to_bank = str(int(parts[3].strip()))
                        from_account = f"{from_bank}-{parts[2].strip()}"
                        to_account = f"{to_bank}-{parts[4].strip()}"
                        amount = float(parts[5].strip())
                        
                        current_pattern.transactions.append({
                            'timestamp': parts[0].strip(),
                            'from_account': from_account, 'to_account': to_account,
                            'amount': amount, 'currency': parts[6].strip(),
                        })
                        current_pattern.accounts_involved.add(from_account)
                        current_pattern.accounts_involved.add(to_account)
                    except (ValueError, IndexError):
                        pass
    
    return patterns


# Parse patterns
laundering_patterns = parse_patterns_file(patterns_file)

# Statistics
pattern_types = {}
for p in laundering_patterns:
    pattern_types[p.pattern_type] = pattern_types.get(p.pattern_type, 0) + 1

print(f"\n{'=' * 60}")
print(f"🔗 LAUNDERING PATTERNS PARSED")
print(f"{'=' * 60}")
print(f"  Total Patterns: {len(laundering_patterns):,}")
for ptype, count in sorted(pattern_types.items(), key=lambda x: -x[1]):
    print(f"    - {ptype:<20} {count:>6,}")
print(f"{'=' * 60}")


## 3. Financial Environment with LangChain Tools

Build the transaction graph with NetworkX and create the `FinancialEnvironment` class with LangChain-compatible tool functions for LangGraph integration.


In [ ]:
# ============================================================================
# FINANCIAL ENVIRONMENT - NetworkX Graph with LangChain Tools
# ============================================================================

from langchain_core.tools import tool

@dataclass 
class FinancialEnvironment:
    """Financial investigation environment with path-validated SAR evaluation."""
    graph: nx.DiGraph = field(default_factory=nx.DiGraph)
    accounts: Dict[str, dict] = field(default_factory=dict)
    laundering_targets: List[str] = field(default_factory=list)
    all_sanctioned: set = field(default_factory=set)
    laundering_destinations: set = field(default_factory=set)
    transitive_illicit: set = field(default_factory=set)
    current_start_account: str = ""
    
    @classmethod
    def from_dataframes(cls, transactions_df: pd.DataFrame, accounts_df: pd.DataFrame) -> 'FinancialEnvironment':
        env = cls()
        
        for _, row in accounts_df.iterrows():
            env.accounts[row['account_id']] = row.to_dict()
            env.graph.add_node(row['account_id'], **row.to_dict())
        
        for _, row in transactions_df.iterrows():
            env.graph.add_edge(
                row['from_account'], row['to_account'],
                transaction_id=row['transaction_id'], amount=row['amount'],
                currency=row.get('currency', 'USD'), timestamp=row['timestamp'],
                is_laundering=row['is_laundering']
            )
        
        env.all_sanctioned = set(accounts_df[accounts_df['is_sanctioned']]['account_id'])
        laundering_txns = transactions_df[transactions_df['is_laundering'] == 1]
        env.laundering_destinations = set(laundering_txns['to_account'].unique())
        env._compute_transitive_illicit()
        env.laundering_targets = list(env.all_sanctioned & env.laundering_destinations)
        
        return env
    
    def _compute_transitive_illicit(self):
        self.transitive_illicit = set()
        laundering_sources = set()
        for u, v, data in self.graph.edges(data=True):
            if data.get('is_laundering', 0) == 1:
                laundering_sources.add(u)
                self.transitive_illicit.add(u)
                self.transitive_illicit.add(v)
        
        for source in laundering_sources:
            visited = {source}
            queue = [source]
            while queue:
                node = queue.pop(0)
                for neighbor in self.graph.successors(node):
                    edge_data = self.graph.edges[node, neighbor]
                    if edge_data.get('is_laundering', 0) == 1 and neighbor not in visited:
                        visited.add(neighbor)
                        self.transitive_illicit.add(neighbor)
                        queue.append(neighbor)
    
    def is_on_laundering_path(self, entity_id: str, max_depth: int = 10) -> bool:
        if not self.current_start_account:
            return False
        try:
            for path in nx.all_simple_paths(self.graph, self.current_start_account, entity_id, cutoff=max_depth):
                if all(self.graph.edges[path[i], path[i+1]].get('is_laundering', 0) == 1 for i in range(len(path)-1)):
                    return True
            return False
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            return False
    
    def get_account_summary_impl(self, account_id: str) -> dict:
        if account_id not in self.accounts:
            return {"error": f"Account {account_id} not found"}
        acc = self.accounts[account_id]
        return {
            "account_id": account_id, "account_type": acc.get('account_type', 'Unknown'),
            "entity_name": acc.get('entity_name', 'Unknown'), "bank_name": acc.get('bank_name', 'Unknown'),
            "risk_score": round(acc.get('risk_score', 0), 2), "is_sanctioned": acc.get('is_sanctioned', False),
            "transitive_illicit": account_id in self.transitive_illicit,
        }
    
    def get_recent_transactions_impl(self, account_id: str, direction: str = "outgoing", limit: int = 5) -> List[dict]:
        if account_id not in self.graph:
            return []
        edges = list(self.graph.out_edges(account_id, data=True) if direction == "outgoing" 
                     else self.graph.in_edges(account_id, data=True))
        edges = sorted(edges, key=lambda e: e[2].get('amount', 0), reverse=True)[:limit]
        
        results = []
        for edge in edges:
            target = edge[1] if direction == "outgoing" else edge[0]
            results.append({
                "counterparty": target, "amount": round(edge[2].get('amount', 0), 2),
                "currency": edge[2].get('currency', 'USD'), "is_laundering": edge[2].get('is_laundering', 0),
                "high_risk_indicator": target in self.transitive_illicit,
            })
        return results
    
    def check_sanctions_list_impl(self, entity_id: str) -> dict:
        is_sanctioned = entity_id in self.all_sanctioned
        return {"entity_id": entity_id, "on_sanctions_list": is_sanctioned, "list_type": "OFAC SDN" if is_sanctioned else None}
    
    def submit_sar_impl(self, entity_id: str, reason: str) -> dict:
        is_sanctioned = entity_id in self.all_sanctioned
        is_primary = entity_id in self.laundering_targets
        on_path = self.is_on_laundering_path(entity_id)
        correct = is_primary or (is_sanctioned and on_path)
        
        if is_primary:
            eval_reason = "PRIMARY_TARGET: sanctioned + receives laundering directly"
        elif is_sanctioned and on_path:
            eval_reason = f"VALID: sanctioned + on laundering path from {self.current_start_account}"
        elif is_sanctioned:
            eval_reason = "INVALID: sanctioned but NOT on laundering path from start"
        else:
            eval_reason = "INVALID: entity is not sanctioned"
        
        return {
            "entity_id": entity_id, "reason": reason, "report_id": f"SAR-{uuid.uuid4().hex[:8].upper()}",
            "correct_identification": correct, "is_sanctioned": is_sanctioned,
            "is_primary_target": is_primary, "on_laundering_path": on_path, "evaluation_reason": eval_reason,
        }
    
    def reset_investigation(self, start_account: str):
        self.current_start_account = start_account


# Build environment
env = FinancialEnvironment.from_dataframes(transactions_df, accounts_df)

print(f"\n{'=' * 60}")
print(f"🏦 FINANCIAL ENVIRONMENT BUILT")
print(f"{'=' * 60}")
print(f"  Graph Nodes:          {env.graph.number_of_nodes():>12,}")
print(f"  Graph Edges:          {env.graph.number_of_edges():>12,}")
print(f"  Sanctioned Accounts:  {len(env.all_sanctioned):>12,}")
print(f"  Primary Targets:      {len(env.laundering_targets):>12,}")
print(f"  Transitive Illicit:   {len(env.transitive_illicit):>12,}")
print(f"{'=' * 60}")


In [ ]:
# ============================================================================
# LANGCHAIN TOOLS - Define tools for LangGraph agent
# ============================================================================

@tool
def get_account_summary(account_id: str) -> str:
    """Get account metadata and risk assessment for an account ID.
    
    Args:
        account_id: The unique account identifier (format: 'bankid-accountnumber')
    
    Returns:
        JSON string with account details including risk_score, is_sanctioned, transitive_illicit
    """
    result = env.get_account_summary_impl(account_id)
    return json.dumps(result, default=str)


@tool
def get_recent_transactions(account_id: str, direction: str = "outgoing") -> str:
    """Get the top-5 recent transactions by amount for an account.
    
    Args:
        account_id: The account to get transactions for
        direction: Either 'outgoing' or 'incoming' to specify transaction direction
    
    Returns:
        JSON string with list of transactions including counterparty, amount, high_risk_indicator
    """
    result = env.get_recent_transactions_impl(account_id, direction)
    return json.dumps(result, default=str)


@tool
def check_sanctions_list(entity_id: str) -> str:
    """Check if an entity is on the OFAC sanctions list.
    
    Args:
        entity_id: The account/entity ID to check against sanctions
    
    Returns:
        JSON string with on_sanctions_list boolean and list_type if sanctioned
    """
    result = env.check_sanctions_list_impl(entity_id)
    return json.dumps(result, default=str)


@tool
def submit_sar(entity_id: str, reason: str) -> str:
    """Submit a Suspicious Activity Report. THIS IS A TERMINAL ACTION.
    
    Only call this after confirming the entity is on the sanctions list!
    
    Args:
        entity_id: The sanctioned entity to report
        reason: Detailed justification for the SAR submission
    
    Returns:
        JSON string with SAR result including correct_identification boolean
    """
    result = env.submit_sar_impl(entity_id, reason)
    return json.dumps(result, default=str)


# Collect all tools
aml_tools = [get_account_summary, get_recent_transactions, check_sanctions_list, submit_sar]
tools_by_name = {tool.name: tool for tool in aml_tools}

print("✓ LangChain tools defined for LangGraph agent:")
for t in aml_tools:
    print(f"  - {t.name}: {t.description[:60]}...")


## 4. LangGraph Agent Architecture

Build the stateful LangGraph agent with:
- **State**: TypedDict holding messages and investigation context
- **Agent Node**: The "brain" that decides the next action
- **Tool Node**: Executes the 4 standard investigation tools
- **Conditional Edge**: Routes between continuing tool usage or triggering final report
- **MemorySaver**: Persistent checkpointing across the investigation loop
- **MLflow Tracing**: Automatic trace logging via LangGraph integration


In [ ]:
# ============================================================================
# LANGGRAPH STATE DEFINITION - Investigation Agent State
# ============================================================================

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage, BaseMessage

# Extended State with investigation-specific fields
# TRAINABLE ARCHITECTURE: State tracks laundering network for model to interpret
class InvestigationState(TypedDict):
    """
    The state for the AML investigation agent.
    
    TRAINABLE DESIGN:
    - State tracks raw information (accounts, transactions, sanctions)
    - Model must learn to interpret this information
    - Model must decide when to submit SAR (not told by system)
    
    Key fields for laundering path tracking:
    - accounts_on_laundering_trail: Set of accounts reachable via is_laundering=1 transactions
    - high_risk_counterparties: Set of accounts with high_risk_indicator=True
    """
    messages: Annotated[List[BaseMessage], operator.add]  # Core message history
    start_account: str                                     # Investigation seed
    accounts_analyzed: Dict[str, dict]                     # Analyzed account data
    entities_checked: Dict[str, bool]                      # Sanctions check results
    investigation_path: List[str]                          # Linear path of exploration (for logging)
    accounts_on_laundering_trail: List[str]               # Accounts reachable via is_laundering=1 (use list for JSON)
    high_risk_counterparties: List[str]                   # Accounts with high_risk_indicator=True
    total_amount_traced: float                             # Total $ traced
    step_count: int                                        # Current step number
    terminated: bool                                       # Whether investigation ended
    success: bool                                          # Whether SAR was correct
    final_result: Optional[dict]                           # Final SAR result


def create_initial_state(start_account: str) -> InvestigationState:
    """Create initial investigation state for a new episode."""
    return {
        "messages": [],
        "start_account": start_account,
        "accounts_analyzed": {},
        "entities_checked": {},
        "investigation_path": [start_account],
        "accounts_on_laundering_trail": [start_account],  # Seed is always on the trail
        "high_risk_counterparties": [],
        "total_amount_traced": 0.0,
        "step_count": 0,
        "terminated": False,
        "success": False,
        "final_result": None,
    }


# System prompt for the investigation agent
INVESTIGATION_SYSTEM_PROMPT = """You are an expert AML (Anti-Money Laundering) investigator. Your task is to investigate financial transaction networks to identify money laundering patterns.

## INVESTIGATION STRATEGY
1. **START**: Get account summary of the seed account to understand its risk profile
2. **EXPLORE**: Get recent transactions to find money flows and counterparties  
3. **FOLLOW**: Investigate high-amount and high-risk counterparties
4. **VERIFY**: Check sanctions list for suspicious entities BEFORE reporting
5. **REPORT**: Submit SAR ONLY after confirming sanctioned status

## IMPORTANT RULES
- ALWAYS check sanctions list before submitting a SAR
- Focus on accounts with high_risk_indicator or transitive_illicit flags
- Follow the money trail by examining outgoing transactions
- Submit SAR only when you have confirmed evidence of sanctions violations

## AVAILABLE TOOLS
- get_account_summary: Get account metadata and risk assessment
- get_recent_transactions: Get top-5 recent transactions by amount
- check_sanctions_list: Verify against OFAC watchlist
- submit_sar: Submit Suspicious Activity Report (TERMINAL ACTION)

Think step-by-step about your investigation strategy before each action."""


print("✓ LangGraph state definition created")
print(f"  - InvestigationState with {len(InvestigationState.__annotations__)} fields")
print(f"  - System prompt: {len(INVESTIGATION_SYSTEM_PROMPT)} characters")


In [ ]:
# ============================================================================
# LOAD MODEL WITH UNSLOTH - Qwen3 MOE with FastModel
# ============================================================================
# NOTE: For MOE models like Qwen3-30B-A3B, use FastModel (not FastLanguageModel)
#       FastModel is imported at the top of the notebook (before transformers)
#       to ensure all Unsloth optimizations are applied correctly.
#
# See: https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune/qwen3-2507

print("📥 Loading Qwen3 MOE model with Unsloth FastModel...")
print(f"   Model: {MODEL_NAME}")
print(f"   Mode:  {'Thinking' if IS_THINKING_MODEL else 'Instruct'}")

# Load model using FastModel (required for MOE architectures)
# Settings from Unsloth docs for Qwen3-2507
model, tokenizer = FastModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=8192,          # Supports up to 256K context, use 8K for memory efficiency
    load_in_4bit=True,            # 4-bit quantization to reduce memory
    load_in_8bit=False,           # Alternative: slightly more accurate but 2x memory
    full_finetuning=False,        # Use LoRA for efficient fine-tuning
    device_map="cuda:0",          # Explicit GPU mapping to prevent CPU offload
    # token = "hf_...",           # Use if accessing gated models
)

# Enable faster inference mode
FastModel.for_inference(model)

print(f"\n{'=' * 60}")
print(f"🤖 MODEL LOADED (Qwen3-2507 MOE)")
print(f"{'=' * 60}")
print(f"  Model:            {MODEL_NAME}")
print(f"  Architecture:     Mixture of Experts (MOE)")
print(f"  Mode:             {'Thinking' if IS_THINKING_MODEL else 'Instruct'}")
print(f"  Max Seq Length:   8,192")
print(f"  Quantization:     4-bit")
print(f"  Device:           {next(model.parameters()).device}")
print(f"  Temperature:      {GENERATION_TEMPERATURE}")
print(f"  Top-P:            {GENERATION_TOP_P}")
print(f"  Top-K:            {GENERATION_TOP_K}")
print(f"{'=' * 60}")


In [ ]:
# ============================================================================
# AGENT NODE - The "Brain" that decides the next action
# ============================================================================
# Uses Qwen3 NATIVE tool calling format for reliable function calling
# Reference: https://github.com/QwenLM/Qwen3/blob/main/docs/source/framework/function_call.md

from langchain_core.runnables import RunnableConfig

# Define tools in Qwen3 native format (OpenAI-compatible JSON Schema)
QWEN3_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_account_summary",
            "description": "Get account metadata and risk assessment for an account ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "account_id": {
                        "type": "string",
                        "description": "The unique account identifier (format: 'bankid-accountnumber')"
                    }
                },
                "required": ["account_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_recent_transactions",
            "description": "Get the top-5 recent transactions by amount for an account.",
            "parameters": {
                "type": "object",
                "properties": {
                    "account_id": {
                        "type": "string",
                        "description": "The account to get transactions for"
                    },
                    "direction": {
                        "type": "string",
                        "enum": ["outgoing", "incoming"],
                        "description": "Transaction direction. Defaults to 'outgoing'"
                    }
                },
                "required": ["account_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_sanctions_list",
            "description": "Check if an entity is on the OFAC sanctions list. ALWAYS call this before submitting SAR!",
            "parameters": {
                "type": "object",
                "properties": {
                    "entity_id": {
                        "type": "string",
                        "description": "The account/entity ID to check against sanctions"
                    }
                },
                "required": ["entity_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "submit_sar",
            "description": "Submit a Suspicious Activity Report. THIS IS A TERMINAL ACTION. Only call after confirming entity is on sanctions list!",
            "parameters": {
                "type": "object",
                "properties": {
                    "entity_id": {
                        "type": "string",
                        "description": "The sanctioned entity to report"
                    },
                    "reason": {
                        "type": "string",
                        "description": "Detailed justification for the SAR submission"
                    }
                },
                "required": ["entity_id", "reason"]
            }
        }
    }
]

class Qwen3AgentWrapper:
    """
    Wrapper to make Qwen3 model compatible with LangGraph using NATIVE tool calling.
    
    This implementation uses Qwen3's built-in function calling format:
    - Tools are passed to apply_chat_template
    - Model outputs tool calls in a structured format
    - Works with both Instruct and Thinking models
    """
    
    def __init__(self, model, tokenizer, tools: List, max_new_tokens: int = 256):
        self.model = model
        self.tokenizer = tokenizer
        self.langchain_tools = tools  # Keep for execution
        self.tool_names = [t.name for t in tools]
        self.max_new_tokens = max_new_tokens
        self.qwen3_tools = QWEN3_TOOLS  # Native format for apply_chat_template
    
    def _build_investigation_context(self, state: dict) -> str:
        """
        Build INFORMATIONAL context for trainable agent.
        
        TRAINABLE DESIGN: Shows raw state data for model to interpret.
        Model must learn to recognize when SAR conditions are met.
        """
        lines = []
        
        # Basic progress
        lines.append(f"Step: {state.get('step_count', 0)}/{MAX_STEPS}")
        lines.append(f"Accounts analyzed: {len(state.get('accounts_analyzed', {}))}")
        
        # Laundering trail info (critical for decision)
        trail = state.get("accounts_on_laundering_trail", [])
        if len(trail) > 1:  # More than just seed
            lines.append(f"Accounts on laundering trail: {len(trail)}")
            lines.append(f"Trail: {' → '.join(trail[-5:])}")
        
        # High-risk counterparties (candidates for sanctions check)
        high_risk = state.get("high_risk_counterparties", [])
        entities_checked = state.get("entities_checked", {})
        unchecked_high_risk = [a for a in high_risk if a not in entities_checked]
        if unchecked_high_risk:
            lines.append(f"High-risk unchecked: {', '.join(unchecked_high_risk[:3])}")
        
        # Sanctions results (critical for SAR decision)
        sanctioned = [e for e, s in entities_checked.items() if s]
        if sanctioned:
            # Show which sanctioned entities are on the laundering trail
            on_trail = [e for e in sanctioned if e in trail]
            not_on_trail = [e for e in sanctioned if e not in trail]
            if on_trail:
                lines.append(f"SANCTIONED ON TRAIL: {', '.join(on_trail)}")
            if not_on_trail:
                lines.append(f"Sanctioned (not on trail): {', '.join(not_on_trail)}")
        
        return " | ".join(lines)
    
    def _build_messages(self, messages: List[BaseMessage], state: dict) -> List[dict]:
        """
        Build INFORMATIONAL messages for trainable agent.
        
        TRAINABLE DESIGN:
        - System prompt explains GOAL and CRITERIA, not specific actions
        - State information is provided, model must interpret
        - Model learns to recognize when to submit SAR
        - No "Submit SAR NOW" directives - model must decide
        """
        formatted = []
        start_account = state.get("start_account", "")
        
        # Gather state information for model to interpret
        trail = state.get("accounts_on_laundering_trail", [])
        high_risk = state.get("high_risk_counterparties", [])
        entities_checked = state.get("entities_checked", {})
        accounts_analyzed = state.get("accounts_analyzed", {})
        
        # Key decision data
        sanctioned = [e for e, s in entities_checked.items() if s]
        sanctioned_on_trail = [e for e in sanctioned if e in trail]
        unchecked_on_trail = [a for a in trail if a not in entities_checked and a != start_account]
        
        # Build context
        context = self._build_investigation_context(state)
        
        # System message - explains GOAL, not specific actions
        system_content = f"""You are an AML investigator finding money laundering.

TARGET ACCOUNT: {start_account}

GOAL: Submit a Suspicious Activity Report (SAR) for an entity that meets BOTH conditions:
1. Entity is on the OFAC sanctions list (check_sanctions_list returns on_sanctions_list=true)
2. Entity is reachable via laundering transactions from {start_account} (is_laundering=1 in transactions)

HOW TO IDENTIFY LAUNDERING PATHS:
- Use get_recent_transactions to find transactions with is_laundering=1
- Accounts reached via is_laundering=1 are on the laundering trail
- high_risk_indicator=true means the counterparty is connected to laundering network

TOOLS:
- get_account_summary(account_id): Get risk info, transitive_illicit flag
- get_recent_transactions(account_id): Find connected accounts, see is_laundering flag
- check_sanctions_list(entity_id): Check if on OFAC list
- submit_sar(entity_id, reason): Submit SAR - ONLY for sanctioned entities on laundering trail

CURRENT STATE:
{context}

AVAILABLE ACCOUNTS: {', '.join(list(accounts_analyzed.keys())[:5]) if accounts_analyzed else start_account}
LAUNDERING TRAIL: {', '.join(trail[-5:]) if trail else start_account}"""
        
        # Add key decision info if available
        if sanctioned_on_trail:
            system_content += f"\n\n⚠️ SANCTIONED ENTITIES ON LAUNDERING TRAIL: {', '.join(sanctioned_on_trail)}"
        elif unchecked_on_trail:
            system_content += f"\n\nAccounts on trail not yet checked for sanctions: {', '.join(unchecked_on_trail[:3])}"
        
        formatted.append({"role": "system", "content": system_content})
        
        # User message
        if not messages or len([m for m in messages if isinstance(m, ToolMessage)]) == 0:
            # First message
            formatted.append({
                "role": "user", 
                "content": f"Investigate account {start_account} for money laundering. Find sanctioned entities on the laundering trail."
            })
        else:
            # Add recent tool results for context
            tool_results = [(msg.name, msg.content) for msg in messages if isinstance(msg, ToolMessage)]
            for name, content in tool_results[-3:]:
                formatted.append({"role": "function", "name": name, "content": content})
            
            # Non-directive follow-up - model must decide
            formatted.append({
                "role": "user",
                "content": "Analyze the results. What is your next action? Consider: Are there sanctioned entities on the laundering trail?"
            })
        
        return formatted
    
    def invoke(self, messages: List[BaseMessage], state: dict = None) -> AIMessage:
        """
        Invoke Qwen3 model with NATIVE tool calling.
        Uses apply_chat_template with tools parameter for proper function calling.
        """
        if state is None:
            state = {"start_account": ""}
        
        # Build messages
        formatted_messages = self._build_messages(messages, state)
        
        # Apply chat template WITH TOOLS for native function calling
        try:
            text = self.tokenizer.apply_chat_template(
                formatted_messages,
                tools=self.qwen3_tools,  # Pass tools for native function calling!
                tokenize=False,
                add_generation_prompt=True,
            )
        except Exception as e:
            # Fallback: some tokenizers don't support tools parameter
            print(f"Warning: apply_chat_template with tools failed: {e}")
            text = self.tokenizer.apply_chat_template(
                formatted_messages,
                tokenize=False,
                add_generation_prompt=True,
            )
        
        # Tokenize
        inputs = self.tokenizer(
            text, 
            return_tensors="pt", 
            truncation=True, 
            max_length=4096
        ).to(self.model.device)
        
        # Generate with correct parameters for Instruct model
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                temperature=GENERATION_TEMPERATURE,
                do_sample=True,
                top_p=GENERATION_TOP_P,
                top_k=GENERATION_TOP_K,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.05,  # Reduce repetition
            )
        
        # Decode response
        response = self.tokenizer.decode(
            outputs[0][inputs.input_ids.shape[1]:], 
            skip_special_tokens=True
        )
        
        # Extract tool calls from response
        tool_calls = self._extract_tool_calls(response)
        
        if tool_calls:
            return AIMessage(content=response, tool_calls=tool_calls)
        else:
            return AIMessage(content=response)
    
    def _extract_tool_calls(self, response: str) -> List[dict]:
        """
        Extract tool calls from Qwen3 response.
        Qwen3 outputs tool calls in various formats - we try all of them.
        """
        tool_calls = []
        
        # Clean up response
        response = response.strip()
        
        # Pattern 1: Qwen3 native <tool_call> tags
        tool_call_pattern = r'<tool_call>\s*(\{.*?\})\s*</tool_call>'
        matches = re.findall(tool_call_pattern, response, re.DOTALL)
        for match in matches:
            try:
                call_data = json.loads(match)
                name = call_data.get("name", "")
                args = call_data.get("arguments", {})
                if isinstance(args, str):
                    args = json.loads(args)
                if name.lower() in [t.lower() for t in self.tool_names]:
                    tool_calls.append({
                        "id": f"call_{uuid.uuid4().hex[:8]}",
                        "name": name.lower(),
                        "args": args
                    })
            except (json.JSONDecodeError, TypeError):
                pass
        
        if tool_calls:
            return tool_calls
        
        # Pattern 2: Raw JSON object with name and arguments
        json_pattern = r'\{[^{}]*"name"\s*:\s*"(\w+)"[^{}]*"arguments"\s*:\s*(\{[^{}]*\})[^{}]*\}'
        matches = re.findall(json_pattern, response, re.DOTALL)
        for name, args_str in matches:
            try:
                args = json.loads(args_str)
                if name.lower() in [t.lower() for t in self.tool_names]:
                    tool_calls.append({
                        "id": f"call_{uuid.uuid4().hex[:8]}",
                        "name": name.lower(),
                        "args": args
                    })
            except json.JSONDecodeError:
                pass
        
        if tool_calls:
            return tool_calls
        
        # Pattern 3: function_call format {"name": ..., "arguments": ...}
        # Try to find any JSON-like structure
        try:
            # Look for JSON object anywhere in response
            json_start = response.find('{')
            if json_start >= 0:
                json_end = response.rfind('}') + 1
                if json_end > json_start:
                    json_str = response[json_start:json_end]
                    data = json.loads(json_str)
                    if isinstance(data, dict) and 'name' in data:
                        name = data['name']
                        args = data.get('arguments', data.get('args', {}))
                        if isinstance(args, str):
                            args = json.loads(args)
                        if name.lower() in [t.lower() for t in self.tool_names]:
                            tool_calls.append({
                                "id": f"call_{uuid.uuid4().hex[:8]}",
                                "name": name.lower(),
                                "args": args
                            })
        except (json.JSONDecodeError, TypeError):
            pass
        
        if tool_calls:
            return tool_calls
        
        # Pattern 4: Direct function call syntax (tool_name(args))
        for tool_name in self.tool_names:
            pattern = rf'\b{tool_name}\s*\(\s*([^)]*)\s*\)'
            matches = re.findall(pattern, response, re.IGNORECASE)
            if matches:
                args = self._parse_args(matches[0])
                if args:  # Only if we got valid args
                    tool_calls.append({
                        "id": f"call_{uuid.uuid4().hex[:8]}",
                        "name": tool_name.lower(),
                        "args": args
                    })
                    return tool_calls
        
        # Pattern 5: Fallback - look for tool name + account ID in response
        # This catches cases where model mentions tool but doesn't format correctly
        start_account = None
        acc_pattern = r'\b(\d{1,3}-\d+)\b'
        acc_matches = re.findall(acc_pattern, response)
        if acc_matches:
            start_account = acc_matches[0]
        
        if start_account:
            # Determine which tool based on keywords
            response_lower = response.lower()
            if 'submit' in response_lower and 'sar' in response_lower:
                tool_calls.append({
                    "id": f"call_{uuid.uuid4().hex[:8]}",
                    "name": "submit_sar",
                    "args": {"entity_id": start_account, "reason": "Suspicious activity detected"}
                })
            elif 'sanction' in response_lower or 'check' in response_lower:
                tool_calls.append({
                    "id": f"call_{uuid.uuid4().hex[:8]}",
                    "name": "check_sanctions_list",
                    "args": {"entity_id": start_account}
                })
            elif 'transaction' in response_lower:
                tool_calls.append({
                    "id": f"call_{uuid.uuid4().hex[:8]}",
                    "name": "get_recent_transactions",
                    "args": {"account_id": start_account}
                })
            elif 'account' in response_lower or 'summary' in response_lower or 'investigate' in response_lower:
                tool_calls.append({
                    "id": f"call_{uuid.uuid4().hex[:8]}",
                    "name": "get_account_summary",
                    "args": {"account_id": start_account}
                })
        
        return tool_calls
    
    def _parse_args(self, args_str: str) -> dict:
        """Parse function arguments from string."""
        args = {}
        if not args_str:
            return args
        
        # Try JSON first
        try:
            return json.loads('{' + args_str + '}')
        except:
            pass
        
        # Handle key="value" or key='value' patterns
        pattern = r'(\w+)\s*=\s*["\']([^"\']+)["\']'
        for match in re.finditer(pattern, args_str):
            args[match.group(1)] = match.group(2)
        
        return args
    


# Create the agent wrapper with Qwen3 native tool calling
agent_model = Qwen3AgentWrapper(model, tokenizer, aml_tools, max_new_tokens=256)

print("✓ Qwen3 Agent Wrapper created (Native Tool Calling)")
print(f"  - Model: {MODEL_NAME}")
print(f"  - Tools: {agent_model.tool_names}")
print(f"  - Max new tokens: {agent_model.max_new_tokens}")
print(f"  - Temperature: {GENERATION_TEMPERATURE}")
print(f"  - Top-P: {GENERATION_TOP_P}")


In [ ]:
# ============================================================================
# LANGGRAPH COMPONENTS - Agent Node, Tool Node, Conditional Edge
# ============================================================================

def agent_node(state: InvestigationState) -> dict:
    """
    The Agent Node - the "brain" that decides the next action.
    
    This node:
    1. Looks at current state (NOT full message history)
    2. Passes structured investigation context to the model
    3. Invokes Qwen3 to decide next action
    4. Returns updated state with new AI message
    
    OPTIMIZATIONS:
    - Passes full state for building concise context
    - State includes: accounts_analyzed, entities_checked, investigation_path
    - Model receives structured summary instead of raw message history
    """
    messages = state["messages"]
    start_account = state["start_account"]
    
    # If no messages yet, add initial user message
    if not messages:
        initial_message = HumanMessage(
            content=f"Begin investigation of account: {start_account}"
        )
        messages = [initial_message]
    
    # Invoke the agent model with FULL STATE (not just start_account)
    # This allows the model to see structured investigation context
    response = agent_model.invoke(messages, state)
    
    # Update step count
    new_step_count = state.get("step_count", 0) + 1
    
    return {
        "messages": [response],
        "step_count": new_step_count,
    }


def tool_node(state: InvestigationState) -> dict:
    """
    The Tool Node - executes tool calls from the agent.
    
    TRAINABLE DESIGN:
    - Tracks accounts_on_laundering_trail (via is_laundering=1 transactions)
    - Tracks high_risk_counterparties (via high_risk_indicator=True)
    - These fields help model learn which accounts are SAR candidates
    
    ERROR HANDLING:
    - Catches exceptions during tool execution
    - Validates tool arguments before calling
    - Returns informative error messages
    """
    messages = state["messages"]
    last_message = messages[-1]
    start_account = state.get("start_account", "")
    
    outputs = []
    updated_accounts = dict(state.get("accounts_analyzed", {}))
    updated_entities = dict(state.get("entities_checked", {}))
    updated_path = list(state.get("investigation_path", []))
    # TRAINABLE: Track laundering trail as list (for JSON serialization)
    updated_laundering_trail = list(state.get("accounts_on_laundering_trail", [start_account]))
    updated_high_risk = list(state.get("high_risk_counterparties", []))
    total_amount = state.get("total_amount_traced", 0.0)
    
    terminated = False
    success = False
    final_result = None
    
    # Handle case where there are no tool calls
    if not hasattr(last_message, 'tool_calls') or not last_message.tool_calls:
        return {
            "messages": [ToolMessage(
                content=json.dumps({"error": "No tool calls found in message"}),
                name="system",
                tool_call_id="no_calls",
            )],
            "accounts_analyzed": updated_accounts,
            "entities_checked": updated_entities,
            "investigation_path": updated_path,
            "accounts_on_laundering_trail": updated_laundering_trail,
            "high_risk_counterparties": updated_high_risk,
            "total_amount_traced": total_amount,
            "terminated": False,
            "success": False,
            "final_result": None,
        }
    
    for tool_call in last_message.tool_calls:
        tool_name = tool_call.get("name", "unknown")
        tool_args = tool_call.get("args", {})
        tool_id = tool_call.get("id", f"call_{uuid.uuid4().hex[:8]}")
        
        try:
            # Normalize tool name (handle case sensitivity)
            tool_name_lower = tool_name.lower()
            matched_tool = None
            for name, tool in tools_by_name.items():
                if name.lower() == tool_name_lower:
                    matched_tool = tool
                    tool_name = name  # Use canonical name
                    break
            
            if matched_tool is None:
                tool_result = json.dumps({
                    "error": f"Unknown tool: {tool_name}",
                    "available_tools": list(tools_by_name.keys())
                })
            else:
                # Validate required arguments
                missing_args = []
                if tool_name == "get_account_summary" and not tool_args.get("account_id"):
                    missing_args.append("account_id")
                elif tool_name == "get_recent_transactions" and not tool_args.get("account_id"):
                    missing_args.append("account_id")
                elif tool_name == "check_sanctions_list" and not tool_args.get("entity_id"):
                    missing_args.append("entity_id")
                elif tool_name == "submit_sar":
                    if not tool_args.get("entity_id"):
                        missing_args.append("entity_id")
                    if not tool_args.get("reason"):
                        # Provide default reason if missing
                        tool_args["reason"] = "Suspicious activity detected during investigation"
                
                if missing_args:
                    tool_result = json.dumps({
                        "error": f"Missing required arguments: {missing_args}",
                        "provided_args": tool_args,
                        "hint": f"Please provide: {', '.join(missing_args)}"
                    })
                else:
                    # Execute the tool
                    tool_result = matched_tool.invoke(tool_args)
                    
                    # Parse result for state updates
                    try:
                        result_dict = json.loads(tool_result)
                        
                        # Update state based on tool type
                        if tool_name == "get_account_summary":
                            acc_id = tool_args.get("account_id", "")
                            if "error" not in result_dict:
                                updated_accounts[acc_id] = result_dict
                                if acc_id and acc_id not in updated_path:
                                    updated_path.append(acc_id)
                        
                        elif tool_name == "get_recent_transactions":
                            source_account = tool_args.get("account_id", "")
                            if isinstance(result_dict, list):
                                for txn in result_dict:
                                    total_amount += txn.get("amount", 0)
                                    counterparty = txn.get("counterparty", "")
                                    
                                    if counterparty:
                                        # Track exploration path
                                        if counterparty not in updated_path:
                                            updated_path.append(counterparty)
                                        
                                        # TRAINABLE: Track laundering trail
                                        # If source is on trail AND this is a laundering txn,
                                        # then counterparty is also on the trail
                                        if txn.get("is_laundering") == 1:
                                            if source_account in updated_laundering_trail or source_account == start_account:
                                                if counterparty not in updated_laundering_trail:
                                                    updated_laundering_trail.append(counterparty)
                                        
                                        # TRAINABLE: Track high-risk counterparties
                                        if txn.get("high_risk_indicator"):
                                            if counterparty not in updated_high_risk:
                                                updated_high_risk.append(counterparty)
                        
                        elif tool_name == "check_sanctions_list":
                            entity_id = tool_args.get("entity_id", "")
                            is_sanctioned = result_dict.get("on_sanctions_list", False)
                            if entity_id:
                                updated_entities[entity_id] = is_sanctioned
                        
                        elif tool_name == "submit_sar":
                            terminated = True
                            success = result_dict.get("correct_identification", False)
                            final_result = result_dict
                    
                    except json.JSONDecodeError:
                        # Tool returned non-JSON, keep as-is
                        pass
        
        except Exception as e:
            # Catch any execution errors
            tool_result = json.dumps({
                "error": f"Tool execution failed: {str(e)}",
                "tool": tool_name,
                "args": tool_args
            })
        
        outputs.append(
            ToolMessage(
                content=tool_result,
                name=tool_name,
                tool_call_id=tool_id,
            )
        )
    
    return {
        "messages": outputs,
        "accounts_analyzed": updated_accounts,
        "entities_checked": updated_entities,
        "investigation_path": updated_path,
        "accounts_on_laundering_trail": updated_laundering_trail,
        "high_risk_counterparties": updated_high_risk,
        "total_amount_traced": total_amount,
        "terminated": terminated,
        "success": success,
        "final_result": final_result,
    }


def should_continue(state: InvestigationState) -> Literal["tools", "end"]:
    """
    The Conditional Edge - the router that decides next step.
    
    This function:
    1. Checks if investigation should terminate
    2. Checks if there are tool calls to execute
    3. Routes to either 'tools' node or 'end'
    """
    # Check termination conditions
    if state.get("terminated", False):
        return "end"
    
    if state.get("step_count", 0) >= MAX_STEPS:
        return "end"
    
    # Check for tool calls in last message
    messages = state.get("messages", [])
    if messages:
        last_message = messages[-1]
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            return "tools"
    
    return "end"


print("✓ LangGraph components defined:")
print("  - agent_node: The brain that decides actions")
print("  - tool_node: Executes investigation tools")
print("  - should_continue: Routes between tools and end")


In [ ]:
# ============================================================================
# BUILD LANGGRAPH - Compile the Investigation Agent Graph
# ============================================================================

import mlflow

# Enable MLflow autologging for LangGraph (via LangChain integration)
mlflow.langchain.autolog()

# Set MLflow experiment
mlflow.set_experiment("AML_Investigation_Agent_LangGraph")
print("✓ MLflow experiment: AML_Investigation_Agent_LangGraph")
print("✓ MLflow autologging enabled for LangGraph")

# Create MemorySaver for persistent checkpointing
memory = MemorySaver()

# Build the StateGraph
builder = StateGraph(InvestigationState)

# Add nodes
builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)

# Set entry point
builder.add_edge(START, "agent")

# Add conditional edge from agent
builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END,
    }
)

# Add edge from tools back to agent (the loop)
builder.add_edge("tools", "agent")

# Compile the graph with MemorySaver
investigation_graph = builder.compile(checkpointer=memory)

print(f"\n{'=' * 60}")
print("🔄 LANGGRAPH INVESTIGATION AGENT COMPILED")
print(f"{'=' * 60}")
print("  Nodes:")
print("    - agent: Qwen3 decision making")
print("    - tools: Tool execution")
print("  Edges:")
print("    - START → agent")
print("    - agent → (conditional) → tools | END")
print("    - tools → agent (loop)")
print("  Checkpointer: MemorySaver (in-memory)")
print("  MLflow: Autologging enabled")
print(f"{'=' * 60}")


In [ ]:
# ============================================================================
# INVESTIGATION EXECUTION - Run Episodes with LangGraph
# ============================================================================

@dataclass
class InvestigationEpisode:
    """Records an investigation episode for evaluation and training."""
    start_account: str
    steps: List[dict] = field(default_factory=list)
    terminated: bool = False
    success: bool = False
    final_result: dict = field(default_factory=dict)
    total_reward: float = 0.0
    
    def add_step(self, tool_name: str, args: dict, result: Any, reward: float = 0.0):
        self.steps.append({
            "step": len(self.steps) + 1, 
            "tool_name": tool_name, 
            "arguments": args, 
            "result": result, 
            "reward": reward
        })
        self.total_reward += reward


def calculate_reward(tool_name: str, tool_args: dict, result: Any, state: dict) -> float:
    """
    TRAINABLE reward function for GRPO.
    
    Rewards are designed to shape learning:
    - Following laundering path: HIGH reward (core skill to learn)
    - Strategic sanctions checks: MEDIUM reward
    - Correct SAR: MAJOR reward (goal)
    - Wrong SAR: MAJOR penalty (avoid false positives)
    - Exploration: Small positive (but less than laundering path)
    """
    reward = -0.05  # Small step penalty (allow exploration)
    
    # Get state for context-aware rewards
    laundering_trail = state.get("accounts_on_laundering_trail", [])
    high_risk = state.get("high_risk_counterparties", [])
    
    if tool_name == "get_account_summary":
        if isinstance(result, dict):
            if result.get("transitive_illicit"):
                reward += 0.3  # Found account on laundering network
            if result.get("is_sanctioned"):
                reward += 0.4  # Found potentially sanctioned (but need trail)
    
    elif tool_name == "get_recent_transactions":
        if isinstance(result, list):
            laundering_count = 0
            high_risk_count = 0
            for txn in result:
                if txn.get("is_laundering") == 1:
                    laundering_count += 1
                if txn.get("high_risk_indicator"):
                    high_risk_count += 1
            
            # KEY REWARD: Finding laundering transactions extends the trail
            reward += 0.4 * laundering_count  # Strong signal for path discovery
            reward += 0.1 * high_risk_count   # Smaller bonus for high-risk
    
    elif tool_name == "check_sanctions_list":
        entity_id = tool_args.get("entity_id", "")
        if isinstance(result, dict) and result.get("on_sanctions_list"):
            # Check if entity is on the laundering trail (strategic check)
            if entity_id in laundering_trail:
                reward += 1.5  # MAJOR: Sanctioned AND on trail = SAR candidate!
            else:
                reward += 0.3  # Found sanctioned but not yet on trail
        elif entity_id in laundering_trail:
            reward += 0.1  # Good strategic check even if not sanctioned
    
    elif tool_name == "submit_sar":
        if isinstance(result, dict):
            if result.get("correct_identification"):
                reward += 5.0  # MAJOR SUCCESS - correct SAR
            else:
                reward -= 3.0  # MAJOR PENALTY - wrong SAR (learn to avoid)
    
    return reward


def run_investigation(
    start_account: str,
    graph = None,
    max_steps: int = MAX_STEPS,
    verbose: bool = False,
    thread_id: str = None,
) -> InvestigationEpisode:
    """
    Run a complete investigation episode using the LangGraph agent.
    
    Args:
        start_account: The seed account to investigate
        graph: The compiled LangGraph (defaults to investigation_graph)
        max_steps: Maximum investigation steps
        verbose: Show detailed step-by-step output (full context, thinking, decisions)
        thread_id: Thread ID for MemorySaver (auto-generated if None)
    
    Returns:
        InvestigationEpisode with full investigation record
    """
    if graph is None:
        graph = investigation_graph
    
    if thread_id is None:
        thread_id = f"investigation-{uuid.uuid4().hex[:8]}"
    
    # Reset environment for this investigation
    env.reset_investigation(start_account)
    
    # Create initial state
    initial_state = create_initial_state(start_account)
    
    # Add initial user message
    initial_state["messages"] = [
        HumanMessage(content=f"Begin investigation of account: {start_account}")
    ]
    
    # Configure with thread ID for MemorySaver and recursion limit
    # Each agent→tools cycle = 2 node invocations, so set limit to 3x MAX_STEPS
    config = {
        "configurable": {"thread_id": thread_id},
        "recursion_limit": MAX_STEPS * 3,
    }
    
    # Create episode recorder
    episode = InvestigationEpisode(start_account=start_account)
    
    if verbose:
        print(f"\n{'═' * 80}")
        print(f"🔍 INVESTIGATION START")
        print(f"{'═' * 80}")
        print(f"  Target Account: {start_account}")
        print(f"  Thread ID:      {thread_id}")
        print(f"  Max Steps:      {max_steps}")
        print(f"  Model:          {MODEL_NAME}")
        print(f"  Temperature:    {GENERATION_TEMPERATURE}")
        print(f"{'═' * 80}")
    
    try:
        # Run the graph
        for step_output in graph.stream(initial_state, config, stream_mode="values"):
            step_count = step_output.get("step_count", 0)
            messages = step_output.get("messages", [])
            
            if verbose:
                # ═══════════════════════════════════════════════════════════════════
                # DETAILED STEP LOGGING - Full context, thinking, and decisions
                # ═══════════════════════════════════════════════════════════════════
                
                print(f"\n{'▓' * 80}")
                print(f"  STEP {step_count}")
                print(f"{'▓' * 80}")
                
                # ─────────────────────────────────────────────────────────────────
                # 1. MEMORY CONTEXT (MemorySaver State)
                # ─────────────────────────────────────────────────────────────────
                print(f"\n┌{'─' * 78}┐")
                print(f"│ 📚 MEMORY CONTEXT (MemorySaver State)                                        │")
                print(f"├{'─' * 78}┤")
                
                # Show all messages in context
                msg_count = len(messages)
                print(f"│ Total Messages in Context: {msg_count}")
                print(f"│")
                
                for idx, msg in enumerate(messages):
                    msg_type = type(msg).__name__
                    if isinstance(msg, HumanMessage):
                        print(f"│ [{idx}] 👤 HumanMessage:")
                        for line in msg.content.split('\n'):
                            print(f"│     {line}")
                    elif isinstance(msg, SystemMessage):
                        print(f"│ [{idx}] ⚙️ SystemMessage:")
                        # Show first 200 chars of system message
                        content_preview = msg.content[:200] + "..." if len(msg.content) > 200 else msg.content
                        for line in content_preview.split('\n'):
                            print(f"│     {line}")
                    elif isinstance(msg, AIMessage):
                        print(f"│ [{idx}] 🤖 AIMessage:")
                        # Show full AI response
                        for line in msg.content.split('\n'):
                            print(f"│     {line}")
                        if msg.tool_calls:
                            print(f"│     [Tool Calls: {len(msg.tool_calls)}]")
                    elif isinstance(msg, ToolMessage):
                        print(f"│ [{idx}] 🔧 ToolMessage ({msg.name}):")
                        # Show full tool response
                        for line in str(msg.content).split('\n'):
                            print(f"│     {line}")
                    print(f"│")
                
                print(f"└{'─' * 78}┘")
                
                # ─────────────────────────────────────────────────────────────────
                # 2. LATEST MESSAGE ANALYSIS
                # ─────────────────────────────────────────────────────────────────
                if messages:
                    last_msg = messages[-1]
                    
                    if isinstance(last_msg, AIMessage):
                        print(f"\n┌{'─' * 78}┐")
                        print(f"│ 🧠 MODEL THINKING & DECISION                                                 │")
                        print(f"├{'─' * 78}┤")
                        
                        # Extract thinking content (between <think> and </think>)
                        content = last_msg.content
                        thinking_match = re.search(r'<think>(.*?)</think>', content, re.DOTALL)
                        
                        if thinking_match:
                            thinking_content = thinking_match.group(1).strip()
                            print(f"│ 💭 THINKING PROCESS:")
                            for line in thinking_content.split('\n'):
                                print(f"│   {line}")
                            print(f"│")
                            
                            # Content after thinking
                            after_think = content.split('</think>')[-1].strip() if '</think>' in content else ""
                            if after_think:
                                print(f"│ 📝 DECISION OUTPUT:")
                                for line in after_think.split('\n'):
                                    print(f"│   {line}")
                        else:
                            # No thinking tags, show full content
                            print(f"│ 📝 FULL RESPONSE (no <think> tags):")
                            for line in content.split('\n'):
                                print(f"│   {line}")
                        
                        print(f"│")
                        
                        # ─────────────────────────────────────────────────────────────────
                        # 3. ROUTING DECISION
                        # ─────────────────────────────────────────────────────────────────
                        print(f"├{'─' * 78}┤")
                        print(f"│ 🚦 ROUTING DECISION                                                          │")
                        print(f"├{'─' * 78}┤")
                        
                        if last_msg.tool_calls:
                            print(f"│ Decision: CONTINUE (tool calls detected)")
                            for tc_idx, tc in enumerate(last_msg.tool_calls):
                                print(f"│")
                                print(f"│ 🔧 TOOL CALL #{tc_idx + 1}:")
                                print(f"│   Tool ID:   {tc.get('id', 'N/A')}")
                                print(f"│   Tool Name: {tc['name']}")
                                print(f"│   Arguments:")
                                args_json = json.dumps(tc['args'], indent=6, default=str)
                                for line in args_json.split('\n'):
                                    print(f"│     {line}")
                                
                                # Check if this is the report tool
                                if tc['name'] == 'submit_sar':
                                    print(f"│   ⚠️ REPORT TOOL DETECTED - Investigation will terminate after this")
                        else:
                            print(f"│ Decision: END (no tool calls)")
                            print(f"│ Agent has finished reasoning without requesting more tools.")
                        
                        print(f"└{'─' * 78}┘")
                    
                    elif isinstance(last_msg, ToolMessage):
                        print(f"\n┌{'─' * 78}┐")
                        print(f"│ 📊 TOOL EXECUTION RESULT                                                     │")
                        print(f"├{'─' * 78}┤")
                        print(f"│ Tool Name: {last_msg.name}")
                        print(f"│ Tool ID:   {getattr(last_msg, 'tool_call_id', 'N/A')}")
                        print(f"│")
                        print(f"│ 📄 FULL TOOL RESPONSE:")
                        
                        # Parse and pretty-print the result
                        try:
                            result = json.loads(last_msg.content) if last_msg.content else {}
                            result_json = json.dumps(result, indent=4, default=str)
                            for line in result_json.split('\n'):
                                print(f"│   {line}")
                        except:
                            for line in str(last_msg.content).split('\n'):
                                print(f"│   {line}")
                        
                        # Calculate and show reward
                        try:
                            result = json.loads(last_msg.content) if last_msg.content else {}
                            # Get tool_args from previous AI message tool calls
                            tool_args = {}
                            for prev_msg in step_output.get("messages", []):
                                if hasattr(prev_msg, 'tool_calls') and prev_msg.tool_calls:
                                    for tc in prev_msg.tool_calls:
                                        if tc.get("name") == last_msg.name:
                                            tool_args = tc.get("args", {})
                                            break
                            reward = calculate_reward(last_msg.name, tool_args, result, step_output)
                            episode.add_step(last_msg.name, tool_args, result, reward)
                            
                            print(f"│")
                            print(f"├{'─' * 78}┤")
                            reward_icon = "🟢" if reward > 0 else ("🔴" if reward < 0 else "⚪")
                            print(f"│ 💰 REWARD CALCULATION: {reward_icon} {reward:+.2f}")
                            print(f"│   Running Total: {episode.total_reward:+.2f}")
                        except Exception as e:
                            print(f"│ ⚠️ Error calculating reward: {e}")
                        
                        print(f"└{'─' * 78}┘")
            
            else:
                # Non-verbose mode: still record steps
                if messages:
                    last_msg = messages[-1]
                    if isinstance(last_msg, ToolMessage):
                        try:
                            result = json.loads(last_msg.content) if last_msg.content else {}
                            # Get tool_args from previous AI message tool calls
                            tool_args = {}
                            for prev_msg in step_output.get("messages", []):
                                if hasattr(prev_msg, 'tool_calls') and prev_msg.tool_calls:
                                    for tc in prev_msg.tool_calls:
                                        if tc.get("name") == last_msg.name:
                                            tool_args = tc.get("args", {})
                                            break
                            reward = calculate_reward(last_msg.name, tool_args, result, step_output)
                            episode.add_step(last_msg.name, tool_args, result, reward)
                        except:
                            pass
            
            # Check for termination
            if step_output.get("terminated", False):
                episode.terminated = True
                episode.success = step_output.get("success", False)
                episode.final_result = step_output.get("final_result", {})
                break
            
            if step_count >= max_steps:
                episode.terminated = True
                break
    
    except Exception as e:
        if verbose:
            print(f"\n{'!' * 80}")
            print(f"❌ ERROR DURING INVESTIGATION")
            print(f"{'!' * 80}")
            print(f"  Exception Type: {type(e).__name__}")
            print(f"  Exception Message: {e}")
            import traceback
            print(f"  Traceback:")
            for line in traceback.format_exc().split('\n'):
                print(f"    {line}")
            print(f"{'!' * 80}")
        episode.terminated = True
    
    if verbose:
        print(f"\n{'═' * 80}")
        print(f"📋 INVESTIGATION COMPLETE")
        print(f"{'═' * 80}")
        print(f"  Result:       {'✅ SUCCESS' if episode.success else '❌ FAILED'}")
        print(f"  Total Steps:  {len(episode.steps)}")
        print(f"  Total Reward: {episode.total_reward:+.2f}")
        if episode.final_result:
            print(f"  SAR Filed:    {'Yes' if episode.final_result.get('filed') else 'No'}")
            print(f"  SAR Reason:   {episode.final_result.get('evaluation_reason', 'N/A')}")
        print(f"{'═' * 80}")
        
        # Summary of all steps
        print(f"\n📊 STEP SUMMARY:")
        for i, step in enumerate(episode.steps):
            reward_icon = "🟢" if step['reward'] > 0 else ("🔴" if step['reward'] < 0 else "⚪")
            print(f"  {i+1}. {step['tool_name']}: {reward_icon} {step['reward']:+.2f}")
    
    return episode


print("✓ Investigation execution function defined")
print("  - run_investigation: Execute full investigation episode with detailed logging")
print("  - calculate_reward: GRPO reward calculation")
print("  - Verbose mode shows: Memory context, Model thinking, Routing decisions, Tool results")


## 5. LLM-as-Judge Evaluation with Gemini

Evaluate agent performance using Gemini as an LLM judge with integrated MLflow tracing.

**Rubric:**
1. **Strategy Quality** (0-10): Did the agent prioritize high-value/high-risk transfers?
2. **Decision Persistence** (0-10): Did it pivot correctly after hitting dead ends?
3. **Outcome** (0-10): Did the agent correctly identify a sanctioned entity?


In [ ]:
# ============================================================================
# LLM-AS-JUDGE EVALUATION - Gemini-based Scoring
# ============================================================================

from google import genai

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
if GEMINI_API_KEY:
    gemini_client = genai.Client(api_key=GEMINI_API_KEY)
    print("✓ Gemini API configured (google.genai)")
else:
    gemini_client = None
    print("⚠️ GEMINI_API_KEY not set - LLM-as-Judge will be disabled")


JUDGE_PROMPT = """You are an expert evaluator of AML investigations.

## Investigation Trace
{trace}

## Evaluation Rubric
1. **Strategy Quality** (0-10): Did the agent prioritize high-value and high-risk transfers?
2. **Decision Persistence** (0-10): Did the agent correctly pivot after hitting dead ends?
3. **Outcome Quality** (0-10): Did the agent correctly identify a sanctioned entity?

## Response Format
Provide your evaluation as JSON:
{{"strategy_score": <0-10>, "persistence_score": <0-10>, "outcome_score": <0-10>, "overall_score": <0-10>, "reasoning": "<brief explanation>"}}
"""


@mlflow.trace(span_type="LLM_JUDGE")
def evaluate_episode_with_llm(episode: InvestigationEpisode) -> dict:
    """Evaluate an investigation episode using Gemini as LLM judge."""
    default_scores = {
        "strategy_score": 0, 
        "persistence_score": 0, 
        "outcome_score": 10 if episode.success else 0,
        "overall_score": 5 if episode.success else 0, 
        "reasoning": "Default score"
    }
    
    if not gemini_client:
        default_scores["reasoning"] = "LLM evaluation disabled (no API key)"
        return default_scores
    
    # Build investigation trace
    trace_lines = [f"Start Account: {episode.start_account}", ""]
    for step in episode.steps:
        tool_name = step.get('tool_name', 'unknown')
        trace_lines.append(f"Step {step['step']}: {tool_name}")
        trace_lines.append(f"  Args: {json.dumps(step.get('arguments', {}), default=str)[:80]}")
        
        result = step.get('result', {})
        if tool_name == "get_account_summary" and isinstance(result, dict):
            trace_lines.append(f"  Result: Type={result.get('account_type', 'Unknown')}, Risk={result.get('risk_score', 0):.2f}, Illicit={result.get('transitive_illicit', False)}")
        elif tool_name == "get_recent_transactions" and isinstance(result, list):
            trace_lines.append(f"  Result: {len(result)} transactions (high_risk: {sum(1 for t in result if t.get('high_risk_indicator'))})")
        elif tool_name == "check_sanctions_list" and isinstance(result, dict):
            trace_lines.append(f"  Result: Sanctioned={result.get('on_sanctions_list', False)}")
        elif tool_name == "submit_sar" and isinstance(result, dict):
            trace_lines.append(f"  Result: Correct={result.get('correct_identification', False)}, Reason={result.get('evaluation_reason', 'N/A')[:50]}")
        else:
            trace_lines.append(f"  Result: {json.dumps(result, default=str)[:100]}")
        trace_lines.append(f"  Reward: {step.get('reward', 0):+.2f}")
    
    trace_lines.append(f"\nFinal: {'SUCCESS' if episode.success else 'FAILED'} | Steps: {len(episode.steps)} | Total Reward: {episode.total_reward:+.2f}")
    
    trace_text = "\n".join(trace_lines)
    
    try:
        response = gemini_client.models.generate_content(
            model=GEMINI_MODEL, 
            contents=JUDGE_PROMPT.format(trace=trace_text)
        )
        
        # Extract JSON from response
        json_match = re.search(r'\{[^{}]*\}', response.text, re.DOTALL)
        if json_match:
            scores = json.loads(json_match.group())
            for key in ["strategy_score", "persistence_score", "outcome_score", "overall_score"]:
                if key not in scores:
                    scores[key] = 0
            if "reasoning" not in scores:
                scores["reasoning"] = "No reasoning provided"
            return scores
        else:
            default_scores["reasoning"] = f"Failed to parse JSON from: {response.text[:100]}"
            return default_scores
            
    except Exception as e:
        default_scores["reasoning"] = f"API error: {str(e)[:50]}"
        return default_scores


print("✓ LLM-as-Judge evaluation configured")
print(f"  - Model: {GEMINI_MODEL}")
print(f"  - Status: {'Enabled' if gemini_client else 'Disabled'}")


In [ ]:
# ============================================================================
# EVALUATION FUNCTION - Run Multiple Episodes
# ============================================================================

def run_evaluation(
    graph,
    stage_name: str, 
    n_episodes: int = EVAL_EPISODES, 
    use_llm_judge: bool = True, 
    verbose: bool = True,
    show_episode_details: bool = True,
) -> pd.DataFrame:
    """
    Run evaluation episodes and collect metrics.
    
    Args:
        graph: The compiled LangGraph to evaluate
        stage_name: Name of this evaluation stage (e.g., "Baseline", "Post-SFT")
        n_episodes: Number of episodes to run
        use_llm_judge: Whether to use Gemini for evaluation
        verbose: Show progress and summaries
        show_episode_details: Show step-by-step for each episode
    
    Returns:
        DataFrame with evaluation results
    """
    # Use pattern seed accounts for guaranteed laundering paths
    eval_seeds = [
        p.seed_account for p in random.sample(
            laundering_patterns, 
            min(n_episodes, len(laundering_patterns))
        ) if p.seed_account
    ]
    
    results = []
    
    print(f"\n{'═' * 70}")
    print(f"🧪 EVALUATION: {stage_name.upper()}")
    print(f"{'═' * 70}")
    print(f"  Episodes: {n_episodes}")
    print(f"  Max Steps: {MAX_STEPS}")
    print(f"  LLM Judge: {'Enabled' if use_llm_judge and gemini_client else 'Disabled'}")
    print(f"{'═' * 70}")
    
    with mlflow.start_run(run_name=f"eval_{stage_name}"):
        for i, seed in enumerate(eval_seeds[:n_episodes]):
            print(f"\n{'▓' * 70}")
            print(f"  EPISODE {i+1}/{n_episodes}")
            print(f"  Seed: {seed}")
            print(f"{'▓' * 70}")
            
            # Run investigation
            episode = run_investigation(
                seed, 
                graph=graph,
                verbose=show_episode_details,
            )
            
            # LLM Judge evaluation
            if use_llm_judge and gemini_client:
                print(f"\n  🤖 LLM-AS-JUDGE EVALUATION...")
                llm_scores = evaluate_episode_with_llm(episode)
                print(f"     Strategy:    {llm_scores.get('strategy_score', 0)}/10")
                print(f"     Persistence: {llm_scores.get('persistence_score', 0)}/10")
                print(f"     Outcome:     {llm_scores.get('outcome_score', 0)}/10")
                print(f"     Overall:     {llm_scores.get('overall_score', 0)}/10")
                if llm_scores.get('reasoning'):
                    print(f"     Reasoning:   {llm_scores.get('reasoning', '')[:60]}...")
            else:
                llm_scores = {
                    "strategy_score": 0, 
                    "persistence_score": 0, 
                    "outcome_score": 10 if episode.success else 0, 
                    "overall_score": 5 if episode.success else 0,
                    "reasoning": "LLM judge disabled"
                }
            
            result = {
                "seed_account": seed, 
                "success": episode.success, 
                "steps": len(episode.steps),
                "total_reward": episode.total_reward, 
                **llm_scores
            }
            results.append(result)
            
            # Episode summary
            print(f"\n  ┌─ EPISODE {i+1} RESULT {'─' * 44}┐")
            print(f"  │ Outcome:      {'✅ SUCCESS' if episode.success else '❌ FAILED':<20} │")
            print(f"  │ Steps:        {len(episode.steps):<20} │")
            print(f"  │ Total Reward: {episode.total_reward:+.2f}{' ' * 17} │")
            print(f"  │ LLM Score:    {llm_scores.get('overall_score', 0)}/10{' ' * 16} │")
            print(f"  └{'─' * 53}┘")
        
        df = pd.DataFrame(results)
        
        # Log to MLflow
        mlflow.log_metrics({
            f"{stage_name}_success_rate": df['success'].mean(),
            f"{stage_name}_avg_steps": df['steps'].mean(),
            f"{stage_name}_avg_reward": df['total_reward'].mean(),
            f"{stage_name}_avg_score": df['overall_score'].mean(),
        })
    
    # Final Summary
    print(f"\n{'═' * 70}")
    print(f"📊 {stage_name.upper()} - FINAL SUMMARY")
    print(f"{'═' * 70}")
    print(f"  Episodes Run:     {len(results)}")
    print(f"  Success Rate:     {df['success'].mean()*100:.1f}% ({df['success'].sum()}/{len(results)})")
    print(f"  Avg Steps:        {df['steps'].mean():.1f}")
    print(f"  Avg Reward:       {df['total_reward'].mean():+.2f}")
    print(f"  Avg LLM Score:    {df['overall_score'].mean():.1f}/10")
    print(f"{'─' * 70}")
    print(f"  Score Breakdown:")
    print(f"    Strategy:       {df['strategy_score'].mean():.1f}/10")
    print(f"    Persistence:    {df['persistence_score'].mean():.1f}/10")
    print(f"    Outcome:        {df['outcome_score'].mean():.1f}/10")
    print(f"{'═' * 70}")
    
    return df


print("✓ Evaluation framework configured")


---

## 6. STAGE 1: Baseline Evaluation (Pre-Training)

Run evaluation on the **base Qwen3 model** before any fine-tuning to establish baseline performance.


In [ ]:
# ============================================================================
# STAGE 1: BASELINE EVALUATION - Pre-Training Performance
# ============================================================================

print("🔍 STAGE 1: BASELINE EVALUATION")
print("   Testing base Qwen3 model (no fine-tuning)")

# Initialize results dictionary for all stages
all_results = {}

# Ensure model is in inference mode
FastModel.for_inference(model)

# Run baseline evaluation
baseline_results = run_evaluation(
    graph=investigation_graph,
    stage_name="Baseline",
    n_episodes=EVAL_EPISODES,
    use_llm_judge=bool(gemini_client),
    verbose=True,
    show_episode_details=True,
)

# Store results for comparison
all_results["Baseline"] = baseline_results

print("\n📊 Baseline Results Summary:")
print(baseline_results[['success', 'steps', 'total_reward', 'overall_score']].describe())


---

## 7. SFT Training Data Generation

Generate Supervised Fine-Tuning samples from laundering patterns with structured tool-calling format.


In [ ]:
# ============================================================================
# SFT TRAINING DATA GENERATION - Multi-turn Conversations
# ============================================================================

def generate_sft_sample(pattern: LaunderingPattern) -> List[dict]:
    """
    Generate a multi-turn SFT training sample with REASONING demonstrations.
    
    TRAINABLE DESIGN:
    - Shows explicit reasoning about laundering path tracking
    - Demonstrates recognition of SAR conditions (sanctioned + on trail)
    - Teaches model to make decisions, not just follow instructions
    """
    messages = []
    seed_account = pattern.seed_account
    terminal_account = pattern.terminal_account
    
    if not seed_account or not terminal_account:
        return []
    
    # Get intermediate accounts from pattern
    intermediate_accounts = list(pattern.accounts_involved - {seed_account, terminal_account})[:2]
    
    # System message
    messages.append({
        "role": "system", 
        "content": INVESTIGATION_SYSTEM_PROMPT + f"\n\nINVESTIGATION TARGET: {seed_account}"
    })
    
    # User starts investigation
    messages.append({
        "role": "user", 
        "content": f"Investigate account {seed_account} for money laundering. Find sanctioned entities on the laundering trail."
    })
    
    # Step 1: Get account summary
    messages.append({
        "role": "assistant", 
        "content": f"""I need to check the seed account's risk profile first.

<tool_call>
{{"name": "get_account_summary", "arguments": {{"account_id": "{seed_account}"}}}}
</tool_call>"""
    })
    
    messages.append({
        "role": "user", 
        "content": f'TOOL RESULT (get_account_summary): {json.dumps({"account_id": seed_account, "account_type": "Corporate", "risk_score": 0.75, "transitive_illicit": True, "is_sanctioned": False})}'
    })
    
    # Step 2: Get transactions - FOCUS ON is_laundering=1
    messages.append({
        "role": "assistant", 
        "content": f"""The account has transitive_illicit=True, meaning it's connected to laundering activity.
Now I need to find the specific laundering transactions (is_laundering=1) to build the laundering trail.

<tool_call>
{{"name": "get_recent_transactions", "arguments": {{"account_id": "{seed_account}", "direction": "outgoing"}}}}
</tool_call>"""
    })
    
    # Show transactions with is_laundering flag
    txn_results = [
        {"counterparty": terminal_account, "amount": pattern.transactions[0]['amount'] if pattern.transactions else 5000, "is_laundering": 1, "high_risk_indicator": True},
        {"counterparty": intermediate_accounts[0] if intermediate_accounts else "999-ACC001", "amount": 3000, "is_laundering": 1, "high_risk_indicator": True},
        {"counterparty": "999-LEGIT01", "amount": 2000, "is_laundering": 0, "high_risk_indicator": False}
    ]
    messages.append({
        "role": "user", 
        "content": f'TOOL RESULT (get_recent_transactions): {json.dumps(txn_results)}'
    })
    
    # Step 3: REASONING about laundering trail before sanctions check
    messages.append({
        "role": "assistant", 
        "content": f"""Analyzing the transactions:
- {terminal_account}: is_laundering=1, high_risk_indicator=True → ON LAUNDERING TRAIL
- {intermediate_accounts[0] if intermediate_accounts else "999-ACC001"}: is_laundering=1 → ON LAUNDERING TRAIL  
- 999-LEGIT01: is_laundering=0 → NOT on laundering trail

For a valid SAR, I need to find an entity that is BOTH:
1. On the sanctions list
2. On the laundering trail (reached via is_laundering=1)

Let me check sanctions for {terminal_account} since it's on the laundering trail.

<tool_call>
{{"name": "check_sanctions_list", "arguments": {{"entity_id": "{terminal_account}"}}}}
</tool_call>"""
    })
    
    messages.append({
        "role": "user", 
        "content": f'TOOL RESULT (check_sanctions_list): {json.dumps({"entity_id": terminal_account, "on_sanctions_list": True, "list_type": "OFAC SDN"})}'
    })
    
    # Step 4: Submit SAR - EXPLICIT REASONING about both conditions
    reason = f"Entity {terminal_account} is sanctioned (OFAC) and on laundering trail from {seed_account}"
    messages.append({
        "role": "assistant", 
        "content": f"""Checking SAR conditions for {terminal_account}:
1. ✓ On sanctions list: YES (OFAC SDN confirmed)
2. ✓ On laundering trail: YES (reached via is_laundering=1 from {seed_account})

BOTH conditions are met. Submitting SAR now.

<tool_call>
{{"name": "submit_sar", "arguments": {{"entity_id": "{terminal_account}", "reason": "{reason}"}}}}
</tool_call>"""
    })
    
    return messages


def generate_sft_dataset(patterns: List[LaunderingPattern], max_samples: int = 100) -> List[dict]:
    dataset = []
    for i, pattern in enumerate(random.sample(patterns, min(max_samples, len(patterns)))):
        messages = generate_sft_sample(pattern)
        if messages:
            dataset.append({"id": f"sft_{i}", "pattern_type": pattern.pattern_type, "messages": messages})
    return dataset


def format_for_unsloth_sft(dataset: List[dict]) -> List[dict]:
    """Format dataset for Unsloth SFT trainer."""
    formatted = []
    for sample in dataset:
        # Apply chat template to messages
        text = tokenizer.apply_chat_template(
            sample['messages'],
            tokenize=False,
            add_generation_prompt=False,
        )
        formatted.append({"text": text})
    return formatted


# Generate SFT dataset
print("📊 Generating SFT training data...")
sft_dataset = generate_sft_dataset(laundering_patterns, max_samples=100)
sft_formatted = format_for_unsloth_sft(sft_dataset)

print(f"✓ Generated {len(sft_formatted)} SFT training samples")


## 8. SFT Training with Unsloth

Fine-tune with LoRA adapters (r=32, alpha=64) targeting attention and MLP layers.


In [ ]:
# ============================================================================
# SFT TRAINING WITH UNSLOTH - LoRA Fine-tuning
# ============================================================================

from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import shutil

# ============================================================================
# CLEANUP - Remove existing PEFT adapters and training artifacts
# ============================================================================

print("🧹 Cleaning up previous training artifacts...")

import gc

# Check if model already has PEFT adapters - if so, user needs to restart kernel
if hasattr(model, 'peft_config') or hasattr(model, 'active_adapter'):
    raise RuntimeError(
        "❌ Model already has PEFT adapters from a previous training run.\n"
        "   Please RESTART THE KERNEL and re-run all cells to train from scratch."
    )
else:
    print("   ✓ Model is clean (no existing adapters)")

# Remove previous training output directories
cleanup_dirs = [
    MODELS_DIR / "sft_output",
    MODELS_DIR / "sft_adapter",
    MODELS_DIR / "grpo_output", 
    MODELS_DIR / "grpo_adapter",
    MODELS_DIR / "aml_agent_final",
]

for dir_path in cleanup_dirs:
    if dir_path.exists():
        try:
            shutil.rmtree(dir_path)
            print(f"   ✓ Removed: {dir_path.name}")
        except Exception as e:
            print(f"   ⚠ Could not remove {dir_path.name}: {e}")

print("✓ Cleanup complete\n")

print("📥 Configuring LoRA adapters for SFT...")

# Get PEFT model with LoRA
model_for_training = FastModel.get_peft_model(
    model, 
    r=LORA_R, 
    target_modules=LORA_TARGET_MODULES, 
    lora_alpha=LORA_ALPHA,
    lora_dropout=0.05, 
    bias="none", 
    use_gradient_checkpointing="unsloth", 
    random_state=RANDOM_SEED,
)

# Create dataset
sft_hf_dataset = Dataset.from_list([{"text": s["text"]} for s in sft_formatted])

# Training arguments
sft_output_dir = MODELS_DIR / "sft_output"
sft_args = TrainingArguments(
    output_dir=str(sft_output_dir), 
    per_device_train_batch_size=2, 
    gradient_accumulation_steps=4,
    num_train_epochs=SFT_EPOCHS, 
    learning_rate=SFT_LEARNING_RATE, 
    warmup_ratio=0.1,
    logging_steps=10, 
    save_strategy="epoch", 
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(), 
    optim="adamw_8bit", 
    seed=RANDOM_SEED, 
    report_to="none",
)

# Create trainer
sft_trainer = SFTTrainer(
    model=model_for_training, 
    tokenizer=tokenizer, 
    train_dataset=sft_hf_dataset,
    args=sft_args, 
    max_seq_length=4096,
)

print(f"\n{'=' * 60}")
print(f"🎓 STARTING SFT TRAINING")
print(f"{'=' * 60}")
print(f"  Epochs: {SFT_EPOCHS} | LR: {SFT_LEARNING_RATE} | LoRA r={LORA_R}")
print(f"{'=' * 60}")

# Train
sft_trainer.train()

# Save adapter
sft_adapter_path = MODELS_DIR / "sft_adapter"
model_for_training.save_pretrained(str(sft_adapter_path))
tokenizer.save_pretrained(str(sft_adapter_path))

print(f"\n✓ SFT training complete! Adapter saved to: {sft_adapter_path}")


## 9. STAGE 2: Post-SFT Evaluation

Evaluate the **SFT-tuned model** to measure improvement from supervised fine-tuning.


In [ ]:
# ============================================================================
# STAGE 2: POST-SFT EVALUATION
# ============================================================================

print("🔍 STAGE 2: POST-SFT EVALUATION")
print("   Testing model after Supervised Fine-Tuning")

# Switch to inference mode
FastModel.for_inference(model_for_training)

# Update agent wrapper with trained model
agent_model_sft = Qwen3AgentWrapper(model_for_training, tokenizer, aml_tools)

# Update agent node to use SFT model
def agent_node_sft(state: InvestigationState) -> dict:
    """Agent node using SFT-trained model with full state context."""
    messages = state["messages"]
    start_account = state["start_account"]
    
    if not messages:
        initial_message = HumanMessage(content=f"Begin investigation of account: {start_account}")
        messages = [initial_message]
    
    # Pass full state for structured context building
    response = agent_model_sft.invoke(messages, state)
    new_step_count = state.get("step_count", 0) + 1
    
    return {"messages": [response], "step_count": new_step_count}

# Rebuild graph with SFT model
builder_sft = StateGraph(InvestigationState)
builder_sft.add_node("agent", agent_node_sft)
builder_sft.add_node("tools", tool_node)
builder_sft.add_edge(START, "agent")
builder_sft.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
builder_sft.add_edge("tools", "agent")

memory_sft = MemorySaver()
investigation_graph_sft = builder_sft.compile(checkpointer=memory_sft)

# Run post-SFT evaluation
post_sft_results = run_evaluation(
    graph=investigation_graph_sft,
    stage_name="Post-SFT",
    n_episodes=EVAL_EPISODES,
    use_llm_judge=bool(gemini_client),
    verbose=True,
    show_episode_details=True,
)

# Store results for comparison
all_results["Post-SFT"] = post_sft_results

# Show improvement over baseline
baseline_sr = all_results["Baseline"]['success'].mean()
post_sft_sr = post_sft_results['success'].mean()
print(f"\n📈 SFT Improvement: {baseline_sr*100:.1f}% → {post_sft_sr*100:.1f}% ({(post_sft_sr-baseline_sr)*100:+.1f}%)")

print("\n📊 Post-SFT Results Summary:")
print(post_sft_results[['success', 'steps', 'total_reward', 'overall_score']].describe())


---

## 10. GRPO Training - Reinforcement Learning

Train with **Group Relative Policy Optimization (GRPO)** using TRL:
- **R_Discovery (+0.5)**: Discovering transitive_illicit nodes
- **R_Logic (+0.3)**: Correct tool sequencing
- **R_Outcome (+2.0)**: Correct SAR submission
- **R_Efficiency (-0.1)**: Step penalty to prevent loops


In [ ]:
# ============================================================================
# GRPO TRAINING - Group Relative Policy Optimization
# ============================================================================

from trl import GRPOTrainer, GRPOConfig

def generate_grpo_prompts(patterns: List[LaunderingPattern], n_prompts: int = 50) -> List[str]:
    """Generate prompts for GRPO training from laundering patterns."""
    prompts = []
    selected = random.sample(patterns, min(n_prompts, len(patterns)))
    
    for pattern in selected:
        seed = pattern.seed_account
        if seed:
            messages = [
                {"role": "system", "content": INVESTIGATION_SYSTEM_PROMPT},
                {"role": "user", "content": f"Begin investigation of account: {seed}"}
            ]
            prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
            )
            prompts.append(prompt)
    
    return prompts


def grpo_reward_function(completions: List[str], prompts: List[str]) -> List[float]:
    """
    GRPO reward function that evaluates model completions.
    """
    rewards = []
    
    for completion, prompt in zip(completions, prompts):
        # Extract seed account from prompt
        match = re.search(r'account:\s*([^\n]+)', prompt)
        if not match:
            rewards.append(-1.0)
            continue
        
        seed_account = match.group(1).strip()
        total_reward = 0.0
        
        # Reset environment
        env.reset_investigation(seed_account)
        
        # Try to extract and execute tool call from completion
        tool_calls = agent_model_sft._extract_tool_calls(completion)
        
        if tool_calls:
            tool_call = tool_calls[0]
            tool_name = tool_call.get("name", "")
            tool_args = tool_call.get("args", {})
            
            if tool_name in tools_by_name:
                result = tools_by_name[tool_name].invoke(tool_args)
                result_dict = json.loads(result)
                total_reward = calculate_reward(tool_name, tool_args, result_dict, {})
            else:
                total_reward = -0.3  # Invalid tool name
        else:
            total_reward = -0.5  # No valid tool call
        
        rewards.append(total_reward)
    
    return rewards


# Generate GRPO training prompts
print("📊 Generating GRPO training prompts...")
grpo_prompts = generate_grpo_prompts(laundering_patterns, n_prompts=50)
grpo_dataset = Dataset.from_dict({"prompt": grpo_prompts})
print(f"✓ Generated {len(grpo_prompts)} GRPO training prompts")

# GRPO Training Configuration
grpo_output_dir = MODELS_DIR / "grpo_output"
grpo_config = GRPOConfig(
    output_dir=str(grpo_output_dir),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=GRPO_EPOCHS,
    learning_rate=GRPO_LEARNING_RATE,
    logging_steps=5,
    save_strategy="epoch",
    report_to="none",
)

print(f"\n{'=' * 60}")
print(f"🎯 STARTING GRPO TRAINING")
print(f"{'=' * 60}")
print(f"  Epochs:        {GRPO_EPOCHS}")
print(f"  Learning Rate: {GRPO_LEARNING_RATE}")
print(f"  Prompts:       {len(grpo_prompts)}")
print(f"{'=' * 60}")

# Create GRPO Trainer
grpo_trainer = GRPOTrainer(
    model=model_for_training,
    config=grpo_config,
    tokenizer=tokenizer,
    train_dataset=grpo_dataset,
    reward_funcs=grpo_reward_function,
)

# Train with GRPO
grpo_trainer.train()

# Save GRPO adapter
grpo_adapter_path = MODELS_DIR / "grpo_adapter"
model_for_training.save_pretrained(str(grpo_adapter_path))
tokenizer.save_pretrained(str(grpo_adapter_path))

print(f"\n✓ GRPO training complete!")
print(f"✓ Adapter saved to: {grpo_adapter_path}")


## 11. STAGE 3: Post-GRPO Evaluation

Evaluate the **GRPO-trained model** to measure improvement from reinforcement learning.


In [ ]:
# ============================================================================
# STAGE 3: POST-GRPO EVALUATION
# ============================================================================

print("🔍 STAGE 3: POST-GRPO EVALUATION")
print("   Testing model after GRPO Reinforcement Learning")

# Switch to inference mode
FastModel.for_inference(model_for_training)

# Update agent wrapper with GRPO-trained model
agent_model_grpo = Qwen3AgentWrapper(model_for_training, tokenizer, aml_tools)

# Update agent node to use GRPO model
def agent_node_grpo(state: InvestigationState) -> dict:
    """Agent node using GRPO-trained model with full state context."""
    messages = state["messages"]
    start_account = state["start_account"]
    
    if not messages:
        initial_message = HumanMessage(content=f"Begin investigation of account: {start_account}")
        messages = [initial_message]
    
    # Pass full state for structured context building
    response = agent_model_grpo.invoke(messages, state)
    new_step_count = state.get("step_count", 0) + 1
    
    return {"messages": [response], "step_count": new_step_count}

# Rebuild graph with GRPO model
builder_grpo = StateGraph(InvestigationState)
builder_grpo.add_node("agent", agent_node_grpo)
builder_grpo.add_node("tools", tool_node)
builder_grpo.add_edge(START, "agent")
builder_grpo.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
builder_grpo.add_edge("tools", "agent")

memory_grpo = MemorySaver()
investigation_graph_grpo = builder_grpo.compile(checkpointer=memory_grpo)

# Run post-GRPO evaluation
post_grpo_results = run_evaluation(
    graph=investigation_graph_grpo,
    stage_name="Post-GRPO",
    n_episodes=EVAL_EPISODES,
    use_llm_judge=bool(gemini_client),
    verbose=True,
    show_episode_details=True,
)

# Store results for comparison
all_results["Post-GRPO"] = post_grpo_results

print("\n📊 Post-GRPO Results Summary:")
print(post_grpo_results[['success', 'steps', 'total_reward', 'overall_score']].describe())


---

## 12. Final Comparison: Baseline vs SFT vs GRPO

Compare metrics across all three training stages:
- **Success Rate**: Percentage of correct SAR submissions
- **Average Steps**: Efficiency of investigation
- **Average Reward**: GRPO reward function score
- **LLM-as-Judge Scores**: Strategy, Persistence, Outcome, Overall


In [ ]:
# ============================================================================
# FINAL COMPARISON - Baseline vs SFT vs GRPO
# ============================================================================

print("\n" + "=" * 80)
print("📊 FINAL COMPARISON: BASELINE vs SFT vs GRPO")
print("=" * 80)

# Build comparison DataFrame
comparison_data = []
for stage_name, results_df in all_results.items():
    comparison_data.append({
        "Stage": stage_name,
        "Success Rate (%)": results_df['success'].mean() * 100,
        "Avg Steps": results_df['steps'].mean(),
        "Avg Reward": results_df['total_reward'].mean(),
        "Avg Strategy Score": results_df['strategy_score'].mean() if 'strategy_score' in results_df else 0,
        "Avg Persistence Score": results_df['persistence_score'].mean() if 'persistence_score' in results_df else 0,
        "Avg Outcome Score": results_df['outcome_score'].mean() if 'outcome_score' in results_df else 0,
        "Avg Overall Score": results_df['overall_score'].mean() if 'overall_score' in results_df else 0,
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.set_index("Stage")

# Display comparison table
print("\n📈 METRICS COMPARISON:")
print("-" * 80)
print(comparison_df.round(2).to_string())
print("-" * 80)

# Calculate improvements
if len(comparison_data) >= 2:
    baseline = comparison_data[0]
    
    print("\n📊 IMPROVEMENTS OVER BASELINE:")
    print("-" * 80)
    
    for i, stage in enumerate(comparison_data[1:], 1):
        stage_name = stage["Stage"]
        success_improvement = stage["Success Rate (%)"] - baseline["Success Rate (%)"]
        reward_improvement = stage["Avg Reward"] - baseline["Avg Reward"]
        score_improvement = stage["Avg Overall Score"] - baseline["Avg Overall Score"]
        
        print(f"\n  {stage_name}:")
        print(f"    Success Rate: {success_improvement:+.1f}%")
        print(f"    Avg Reward:   {reward_improvement:+.2f}")
        print(f"    Overall Score: {score_improvement:+.1f}/10")

print("\n" + "=" * 80)

# Log to MLflow
with mlflow.start_run(run_name="final_comparison"):
    for stage_name, results_df in all_results.items():
        mlflow.log_metrics({
            f"{stage_name}_success_rate": results_df['success'].mean(),
            f"{stage_name}_avg_steps": results_df['steps'].mean(),
            f"{stage_name}_avg_reward": results_df['total_reward'].mean(),
            f"{stage_name}_avg_overall_score": results_df.get('overall_score', pd.Series([0])).mean(),
        })

print("✓ Comparison logged to MLflow")


In [ ]:
# ============================================================================
# VISUALIZATION - Performance Comparison Charts
# ============================================================================

import matplotlib.pyplot as plt

# Create comparison visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('AML Investigation Agent (LangGraph): Training Stage Comparison', fontsize=14, fontweight='bold')

stages = list(all_results.keys())
colors = ['#e74c3c', '#3498db', '#2ecc71'][:len(stages)]

# 1. Success Rate
ax1 = axes[0, 0]
success_rates = [all_results[s]['success'].mean() * 100 for s in stages]
bars1 = ax1.bar(stages, success_rates, color=colors, edgecolor='black', linewidth=1.2)
ax1.set_ylabel('Success Rate (%)', fontweight='bold')
ax1.set_title('SAR Submission Accuracy', fontweight='bold')
ax1.set_ylim(0, 100)
for bar, val in zip(bars1, success_rates):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.1f}%', 
             ha='center', va='bottom', fontweight='bold')

# 2. Average Reward
ax2 = axes[0, 1]
avg_rewards = [all_results[s]['total_reward'].mean() for s in stages]
bars2 = ax2.bar(stages, avg_rewards, color=colors, edgecolor='black', linewidth=1.2)
ax2.set_ylabel('Average Reward', fontweight='bold')
ax2.set_title('GRPO Reward Score', fontweight='bold')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
for bar, val in zip(bars2, avg_rewards):
    ypos = bar.get_height() + 0.1 if val >= 0 else bar.get_height() - 0.3
    ax2.text(bar.get_x() + bar.get_width()/2, ypos, f'{val:.2f}', 
             ha='center', va='bottom' if val >= 0 else 'top', fontweight='bold')

# 3. Average Steps
ax3 = axes[1, 0]
avg_steps = [all_results[s]['steps'].mean() for s in stages]
bars3 = ax3.bar(stages, avg_steps, color=colors, edgecolor='black', linewidth=1.2)
ax3.set_ylabel('Average Steps', fontweight='bold')
ax3.set_title('Investigation Efficiency', fontweight='bold')
for bar, val in zip(bars3, avg_steps):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{val:.1f}', 
             ha='center', va='bottom', fontweight='bold')

# 4. LLM-as-Judge Overall Score
ax4 = axes[1, 1]
if 'overall_score' in all_results[stages[0]].columns:
    overall_scores = [all_results[s]['overall_score'].mean() for s in stages]
    bars4 = ax4.bar(stages, overall_scores, color=colors, edgecolor='black', linewidth=1.2)
    ax4.set_ylabel('Overall Score (0-10)', fontweight='bold')
    ax4.set_title('LLM-as-Judge Score', fontweight='bold')
    ax4.set_ylim(0, 10)
    for bar, val in zip(bars4, overall_scores):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, f'{val:.1f}', 
                 ha='center', va='bottom', fontweight='bold')
else:
    ax4.text(0.5, 0.5, 'LLM-as-Judge\nNot Available', ha='center', va='center', fontsize=12)
    ax4.set_title('LLM-as-Judge Score', fontweight='bold')

plt.tight_layout()
comparison_chart_path = OUTPUT_DIR / "langgraph_training_comparison.png"
plt.savefig(str(comparison_chart_path), dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Comparison chart saved to {comparison_chart_path}")


---

## 13. Save Final Model

Save the trained model adapters for deployment.


In [ ]:
# ============================================================================
# SAVE FINAL MODEL
# ============================================================================

# Save final adapter to models directory
final_model_path = MODELS_DIR / "aml_agent_langgraph_final"
model_for_training.save_pretrained(str(final_model_path))
tokenizer.save_pretrained(str(final_model_path))

# Save comparison results
comparison_csv_path = OUTPUT_DIR / "langgraph_training_comparison.csv"
comparison_df.to_csv(str(comparison_csv_path))

print(f"\n{'=' * 60}")
print(f"✅ TRAINING COMPLETE")
print(f"{'=' * 60}")
print(f"  Final Model:      {final_model_path}")
print(f"  SFT Adapter:      {sft_adapter_path}")
print(f"  GRPO Adapter:     {grpo_adapter_path}")
print(f"  Comparison Chart: {OUTPUT_DIR / 'langgraph_training_comparison.png'}")
print(f"  Comparison CSV:   {comparison_csv_path}")
print(f"{'=' * 60}")

print(f"\n🎉 AML Investigation Agent (LangGraph) Training Complete!")
print(f"\n📊 Final Performance Summary:")
print(comparison_df.round(2).to_string())

print(f"\n🔗 LangGraph Architecture Summary:")
print(f"  - State: InvestigationState with messages + investigation context")
print(f"  - Agent Node: Qwen3 decision making with tool calling")
print(f"  - Tool Node: Executes 4 AML investigation tools")
print(f"  - Conditional Edge: Routes between tools and end")
print(f"  - Checkpointer: MemorySaver for persistent state")
print(f"  - Observability: MLflow autologging for full traces")

print(f"\nNext Steps:")
print(f"  1. Load adapter with Unsloth for inference")
print(f"  2. Deploy agent with LangGraph orchestration")
print(f"  3. Monitor with MLflow tracing UI")
